In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

from utils_alertas import *

EJEMPLO:
202104 mes cerrado de la cosecha de 202104 [202011, 202012, 202101, 202102, 202103, 202104] [202102, 202103, 202104] 
 historia 
 [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104] 
 ---------------------------------------------------------------------------------------------------- 
 [202105, 202106] 
 ---------------------------------------------------------------------------------------------------- 
 [202005, 202006]


In [2]:
filtro_banca_s2provincia

<function utils_alertas.filtro_banca_s2provincia(df, col_funnel='FUNNEL_4', cosechas_a_analizar=['202009', '202010', '202011', '202012', '202101', '202102'])>

In [3]:
#nativos
import os
import unicodedata

#terceros
import pandas as pd
#import pymssql
#import missingno as msno
from collections import Counter
import scorecardpy as sc

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

In [4]:
data = pd.read_csv('AVANCE_5.5.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,PERIODO,COD_UNICO,RUC,FLG_VIGENTE,NOMBRE_COMPLETO,FLG_SUSPENDIDO,FLG_VENCIDA,FLG_CANCELADO,FLG_MONOLINEA,FLG_SOLO_REACTIVA,...,target_fusion_atraso_feve,PERIODO_ZONAL,target_FINAL_fusion_atraso_feve_2Mbueno,target_FINAL_fusion_atraso_SEGUIDO_15000_FEVE_2Mbueno,target_fusion_atraso_feve_3_3Mbueno,target_fusion_atraso_feve_3_1Mbueno,target_fusion_atraso_feve_3,target_variante_atraso_feve_3_3Mbueno,target_variante_atraso_feve_3_1Mbueno,target_variante_atraso_feve
0,202008,0002225225,20170717261,1,RUMI IMPORT,0,0,0,0,0,...,0.0,202008_BEL ZONAL 3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,202008,0004039624,10010600338,0,REATEGUI FLORES LUCAS,0,1,0,0,0,...,1.0,202008_BEP ZONAL 2,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN
2,202008,0004017117,20293331066,1,PRECISION PERU,1,1,1,0,0,...,NaN,202008_BEL ZONAL 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202008,0003980042,20101391397,0,COMERCIAL INDUSTRIAL DELTA,1,0,1,0,1,...,NaN,202008_BEL ZONAL 1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
4,202008,0003290442,20100244129,1,AGRICOLA COMERCIAL E INDUSTRIAL,1,0,0,0,1,...,0.0,202008_BEL ZONAL 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data.shape

(24014, 2659)

## DATA A ANALIZAR

In [6]:
ss = pd.read_excel('CLIENTES_BEL_BEP.xlsx', dtype={
    'CODUNICOCLI': str
}, engine='openpyxl')
ss.head(2)

,CODUNICOCLI,TIPDOC,CODDOC,NOMBRE_COMPLETO,SEGMENTO,ZONAL,ENCARGADO,NOMBRE_JEFE,DES_GRUPO_ECONOMICO,ACTIVIDAD,...,CIIU,DESCP_CIIU,SECTOR_PERSONALIZADO,SEI_MCDO,DIRECTAS,INDIRECTAS,REACTIVA,DEPOSITOS,RIESGO_MAXIMO_CLIENTE,RIESGO_MAXIMO_GRUPO
0,0000057091,2,20131529008,SEKUR PERU,S2,BEL ZONAL 2,AISSA LIZBETH TRONCOS RIOFRIO,ROBERTO CARLOS PALACIOS PANDO,GABUTEAU BARBA,NaN,...,5239,Venta al por menor de otros productos en almac...,Comercio Minorista,COMERCIO,0.0,0.0,0.0,705486.5557,NaN,NaN
1,0000379289,2,20100180210,FARMAGRO,S1,BEL ZONAL 3,PATRICIA VALENZUELA,NaN,FARMAGRO,NaN,...,5149,Venta al por mayor de otros productos intermed...,Com. Mayor Bs. Interm. y de Cap.,COMERCIO,0.0,0.0,9324321.0,89964.8666,5543753.0,5543753.0


In [7]:
ss['COD_UNICO'] = ss['CODUNICOCLI'].copy()
ss.shape, ss['CODUNICOCLI'].unique().shape

((9302, 22), (9293,))

In [8]:
ss = ss.drop_duplicates(subset=['COD_UNICO', 'SEGMENTO'])
ss.shape

(9293, 22)

In [9]:
data = data.merge(
    ss[['SEGMENTO', 'COD_UNICO']], how='left', on=['COD_UNICO']
)
data['SEGMENTO'].value_counts(dropna=False)

S2     15625
S1      6614
NaN     1775
Name: SEGMENTO, dtype: int64

In [10]:
data[data['SEGMENTO'].isnull()]['ZONAL'].value_counts(dropna=False)

BC GRUPO 2     880
BC GRUPO 1     751
BEL ZONAL 2     75
BEP ZONAL 1     28
BEP ZONAL 2     22
BEP ZONAL 3     15
BEL ZONAL 1      3
BEL ZONAL 3      1
Name: ZONAL, dtype: int64

In [11]:
data[
    (data['SEGMENTO'].isnull()) &  (data['COD_UNICO'] == '0015872383')
]

,PERIODO,COD_UNICO,RUC,FLG_VIGENTE,NOMBRE_COMPLETO,FLG_SUSPENDIDO,FLG_VENCIDA,FLG_CANCELADO,FLG_MONOLINEA,FLG_SOLO_REACTIVA,...,PERIODO_ZONAL,target_FINAL_fusion_atraso_feve_2Mbueno,target_FINAL_fusion_atraso_SEGUIDO_15000_FEVE_2Mbueno,target_fusion_atraso_feve_3_3Mbueno,target_fusion_atraso_feve_3_1Mbueno,target_fusion_atraso_feve_3,target_variante_atraso_feve_3_3Mbueno,target_variante_atraso_feve_3_1Mbueno,target_variante_atraso_feve,SEGMENTO
295,202008,0015872383,20600537211,0,MESA PERU,0,0,0,0,0,...,202008_BEP ZONAL 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pd.crosstab(
    data['ZONAL'].fillna(-1),  data['SEGMENTO'].fillna(-1)
)

SEGMENTO,-1,S1,S2
ZONAL,,,
BC GRUPO 1,751,0,0
BC GRUPO 2,880,4,0
BEL ZONAL 1,3,2488,2
BEL ZONAL 2,75,0,11041
BEL ZONAL 3,1,1887,0
BEP ZONAL 1,28,943,1675
BEP ZONAL 2,22,605,1570
BEP ZONAL 3,15,687,1337


In [13]:
import pymssql
from utils_alertas import *
#from querys import *
try:
    connection_market.close()
except:
    pass

#conneciom MARKET
connection_market = pymssql.connect(**dicc_conection_market)

sql_ = """
   SELECT * FROM ODS.HD_VMVP_BASE_ALERTA_CARTERA_BE WITH (NOLOCK)
"""

univ = pd.read_sql(
   sql_ , connection_market
)
connection_market.close()
univ.head(3)

,FEVE,PERIODO,CODUNICOCLI,NOMBRE_COMPLETO,FLG_VIGENTE,FLG_SUSPENDIDO,FLG_VENCIDA,FLG_CANCELADO,FLG_MONOLINEA,FLG_SOLO_REACTIVA,...,CLASIF_SBS,CLASIF_IBK,DEPARTAMENTO,FEC_INGRESO,FEC_PRIMER_PRODUCTO_APERTURADO,SEI_MCDO,TIPDOC,CODDOC,NUM_RUC,SEGMENTO_CLIENTE
0,None,202008,0002225225,RUMI IMPORT,1,0,0,0,0,0,...,NORMAL,NORMAL,CALLAO,1994-06-22,1950-01-01,CONSTRUCCIÓN,2,20170717261,20170717261,S1
1,EX-FEVE,202008,0004039624,REATEGUI FLORES LUCAS,0,0,1,0,0,0,...,NORMAL,NORMAL,SAN MARTIN,1995-09-22,1950-01-01,COMERCIO,1,01060033,10010600338,S2
2,SEGUIR,202008,0004017117,PRECISION PERU,1,1,1,1,0,0,...,NORMAL,NORMAL,LIMA,1995-09-15,1995-10-18,COMERCIO,2,20293331066,20293331066,S1


In [14]:
print(data.shape)
data['CODUNICOCLI'] = data['COD_UNICO']
data = data.merge(univ[['PERIODO', 'CODUNICOCLI', 'SEGMENTO_CLIENTE']], how='left', on=['PERIODO', 'CODUNICOCLI'])
del univ
print(data.shape)

(24014, 2660)
(24014, 2662)


In [15]:
data['SEGMENTO'] = data['SEGMENTO'].combine_first(data['SEGMENTO_CLIENTE'])

pd.crosstab(
    data['ZONAL'].fillna(-1),  data['SEGMENTO'].fillna(-1)
)

SEGMENTO,,S1,S2
ZONAL,,,
BC GRUPO 1,751,0,0
BC GRUPO 2,880,4,0
BEL ZONAL 1,0,2491,2
BEL ZONAL 2,0,0,11116
BEL ZONAL 3,0,1888,0
BEP ZONAL 1,0,943,1703
BEP ZONAL 2,0,606,1591
BEP ZONAL 3,0,690,1349


In [16]:
#data['GRUPO_ECON_FEVE_DURO'].head()

In [17]:
col_no_review = list(set(['COD_UNICO', 'PERIODO', 'DES_GRUPO_ECONOMICO', 'ENCARGADO', 
                 'NOMBRE_COMPLETO', 'NOMBRE_JEFE', 'RUC', 'GRUPO_ECON_FEVE_SEGUIR', 'MTO_CASH_IN', 'DESC_ATRASO_SF',
                 'GRUPO_ECON_CLASIF_NO_NORMAL', 'GRUPO_ECON_FEVE_DURO', 'RFMClass_septil', 'MTO_CASH_IN', 'DESC_ATRASO_SF', 'CODUNICOCLI',
                 'SUBSECTOR'] + [col for col in data.columns if 'DESCR' in col.upper()]))
col_no_review = list(set(col_no_review))
len(col_no_review)

21

In [18]:
all_data = data.copy()

In [19]:
print(data.shape)
data = filtro_banca_s2provincia(data)
print(data.shape)

(24014, 2662)
(2082, 2662)


In [20]:
data['PERIODO'].value_counts(dropna=False)

202009    359
202010    355
202011    352
202012    350
202102    333
202101    333
Name: PERIODO, dtype: int64

In [21]:
data['ZONAL'].value_counts(dropna=False)

BEP ZONAL 1    853
BEP ZONAL 2    689
BEP ZONAL 3    540
Name: ZONAL, dtype: int64

In [22]:
data['SEGMENTO'].value_counts(dropna=False)

S2    2082
Name: SEGMENTO, dtype: int64

#### REVISION DE NO NULOS

In [23]:
col_target = 'target_FINAL_fusion_atraso_feve_2Mbueno'

In [24]:
data[data[col_target].notnull()].shape, data.shape

((1383, 2662), (2082, 2662))

In [25]:
nn = data[data[col_target].notnull()].isnull().sum()
no_nulos = list(nn[nn == 0].index)
len(no_nulos)

447

In [26]:
no_nulos.remove('PERIODO') 
no_nulos.remove('COD_UNICO')

## Peso de la evidencia  / informacion de su valor

In [27]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.max_columns", 1000)

In [28]:
#iv_yoy = sc.iv(
#    data[data[col_target].notnull()].drop(
#        [col for col in data.columns if 'target' in col and col != col_target] + ['COD_UNICO', 'PERIODO'], axis=1), y=col_target
#)

In [29]:
#select = iv_yoy[
#    (iv_yoy['info_value'] > 0.1) #&  (iv_yoy['info_value'] <= 1)
#]
#select.shape

In [30]:
#dict_iv = select.set_index('variable').to_dict().get('info_value', {})

In [31]:
#select[:55]

In [32]:
#select[55:110]

In [33]:
#select[110:165]

In [34]:
#select[165:220]

In [35]:
for cc in ['RFMClass_quartil', 'FEC_INGRESO'] + [col for col in data.columns if 'listado' in col.lower()]:
    if cc in data.columns:
        del data[cc]
    
    if cc in no_nulos:
        no_nulos.remove(cc)

In [36]:
%%time

cortes = sc.woebin(
    data[
        (data[col_target].notnull())
    ].drop(
        [col for col in data.columns if 'target' in col and col != col_target] + col_no_review, 
        axis=1
    ),
    y=col_target,
    method='tree',
    no_cores=4,
    count_distr_limit=0.05  # el 5% por bin es lo mínimo recomendable para arboles
)

[INFO] creating woe binning ...
Binning on 1383 rows and 2536 columns in 00:14:05
Wall time: 14min 5s


In [37]:
dic_maxprob_corte = {}
for k, v in cortes.items():
    #print(k, type(v))
    ##print(">"*20, ' ', k.upper())
    #display(v)
    #dic_maxprob_corte[k] = v['badprob'].max()
    max_probabilidad_malo = v[v['bin'] != 'missing']['badprob'].max()
    informacion_valor = v['total_iv'].max()
    dic_maxprob_corte[k] = [
        max_probabilidad_malo, informacion_valor, (max_probabilidad_malo + informacion_valor) / 2
    ]

In [38]:
listado_mayor_prob_corte = sorted([(k, v) for k, v in dic_maxprob_corte.items()], key=lambda _: _[1][1], reverse=True)
listado_mayor_prob_corte[:250]

[('DEPARTAMENTO',
  [0.19653179190751446, 0.6052329339912047, 0.4008823629493596]),
 ('FM_category_septil',
  [0.13709677419354838, 0.5016791482906326, 0.3193879612420905]),
 ('GAR_TOT_PREFERIDAS_SF',
  [0.22727272727272727, 0.4615747460386107, 0.344423736655669]),
 ('COSTO_VENTAS_ANUAL_CEF_SITUACION',
  [0.1409090909090909, 0.4600358271361452, 0.30047245902261804]),
 ('AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT',
  [0.2, 0.3985340583904179, 0.299267029195209]),
 ('TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1',
  [0.21428571428571427, 0.39522938632395527, 0.30475755030483476]),
 ('PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V',
  [0.152317880794702, 0.37820085305744633, 0.26525936692607416]),
 ('PERCETN_DEBT_IN__IBKSF_ULT_RCC',
  [0.152317880794702, 0.37820085305744633, 0.26525936692607416]),
 ('NRO_DOCS_LAB_U1M',
  [0.20270270270270271, 0.3781546826958602, 0.29042869269928145]),
 ('NRO_DOCS_LAB_U12M',
  [0.20207253886010362, 0.37199010232925517, 0.2870313205946794]),
 ('NRO_PROD_OTRA_ENTIDAD_

In [39]:
vars_top = [_[0] for _ in listado_mayor_prob_corte[:200]]
len(vars_top)

200

In [40]:
if 'FEC_INGRESO' in no_nulos:
    no_nulos.remove('FEC_INGRESO')

## Correlacion lineal

In [41]:
data_corr = data[data[col_target].notnull()][no_nulos].corr()

In [42]:
data_corr[col_target].sort_values(ascending=False).apply(lambda _: abs(_))[:58]

target_variante_atraso_feve_3_1Mbueno                    1.000000
target_fusion_atraso_feve_3_1Mbueno                      1.000000
target_FINAL_fusion_atraso_feve_2Mbueno                  1.000000
target_SF_8_dias_atraso_o_mas_2Mbueno                    0.915454
target_SF_8_dias_atraso_o_mas_1Mbueno                    0.915454
target_fusion_atraso_15_feve_all_2Mbueno                 0.845702
target_SF_15_dias_atraso_o_mas_2Mbueno                   0.749079
target_rcc_8_dias_atraso_1Mbueno                         0.702838
target_rcc_8_dias_atraso_2Mbueno                         0.702838
target_8_dias_atraso_ibk_1Mbueno                         0.647624
target_8_dias_atraso_ibk_2Mbueno                         0.647624
target_rcc_15_dias_atraso_2Mbueno                        0.641245
target_CONTRASTE_30_dias_atraso                          0.552642
target_15_dias_atraso_1Mbueno                            0.474789
target_15_dias_atraso_2Mbueno                            0.474789
target_CON

In [43]:
#[col for col in data.columns if 'TOTAL' in col.upper()]

In [44]:
#select_1.to_excel("VARIABLES_CANDIDATAS_SEGUIR_COVID.xlsx", index=False)

### NUEVAS VARIABLES

Los cortes nuevos los replicamos a toda la data

In [45]:
data = all_data.copy()
del all_data

In [46]:
data = add_new_var_x_cut_up(data, colum='M_Septile', cortes=[5], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='F_Septile', cortes=[5], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='M_Septile', cortes=[6], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='F_Septile', cortes=[6], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='M_Septile', cortes=[7], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='F_Septile', cortes=[7], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='F_Quartile', cortes=[4], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='M_Quartile', cortes=[4], impute=0, up=True, soft=False)

>>  _5_cut_up_M_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4457943925233645  PRECISION:  0.060257705912076805  F1:  0.10616514578232807  GINI:  0.02230583508285444
//////////////////////////////////////////////////
>>  _5_cut_up_F_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.44672897196261685  PRECISION:  0.059757469683710464  F1:  0.10541404785533136  GINI:  0.01857230567547119
//////////////////////////////////////////////////
>>  _6_cut_up_M_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.28785046728971964  PRECISION:  0.059128431560760225  F1:  0.09810479375696768  GINI:  0.008845571468246272
//////////////////////////////////////////////////
>>  _6_cut_up_F_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.29439252336448596  PRECISION:  0.05831173639392818  F1:  0.0973423980222497  GINI:  0.004798990403083181
//////////////////////////////////////////////////
>>  _7_cut_up_M_Septi

In [47]:
var = vars_top[0]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEPARTAMENTO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEPARTAMENTO,"APURIMAC%,%PUNO%,%CUSCO%,%TUMBES%,%MADRE DE DIOS%,%ICA%,%CAJAMARCA",222,0.160521,220,2,0.009009,-2.346727,0.369661,0.605233,"APURIMAC%,%PUNO%,%CUSCO%,%TUMBES%,%MADRE DE DIOS%,%ICA%,%CAJAMARCA",False
1,DEPARTAMENTO,"LAMBAYEQUE%,%AREQUIPA%,%LIMA%,%TACNA",402,0.290672,379,23,0.057214,-0.448289,0.048600,0.605233,"LAMBAYEQUE%,%AREQUIPA%,%LIMA%,%TACNA",False
2,DEPARTAMENTO,"UCAYALI%,%JUNIN",177,0.127983,161,16,0.090395,0.044938,0.000263,0.605233,"UCAYALI%,%JUNIN",False
3,DEPARTAMENTO,"HUANUCO%,%LA LIBERTAD%,%AYACUCHO%,%SAN MARTIN",409,0.295734,364,45,0.110024,0.263262,0.022850,0.605233,"HUANUCO%,%LA LIBERTAD%,%AYACUCHO%,%SAN MARTIN",False
4,DEPARTAMENTO,"LORETO%,%ANCASH%,%PIURA%,%AMAZONAS%,%MOQUEGUA",173,0.125090,139,34,0.196532,0.945640,0.163859,0.605233,"LORETO%,%ANCASH%,%PIURA%,%AMAZONAS%,%MOQUEGUA",False


In [48]:
var = vars_top[1]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FM_category_septil


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FM_category_septil,"_11%,%_12%,%_13%,%_14%,%_15%,%_21%,%_74%,%_63%,%_57%,%_47%,%_26%,%_31%,%_41%,%_37%,%_43%,%_65",208,0.150398,206,2,0.009615,-2.280976,0.334019,0.501679,"_11%,%_12%,%_13%,%_14%,%_15%,%_21%,%_74%,%_63%,%_57%,%_47%,%_26%,%_31%,%_41%,%_37%,%_43%,%_65",False
1,FM_category_septil,"_23%,%_33%,%_32%,%_46",147,0.106291,141,6,0.040816,-0.803247,0.049511,0.501679,"_23%,%_33%,%_32%,%_46",False
2,FM_category_septil,"_44%,%_66%,%_52%,%_56%,%_67%,%_35%,%_55%,%_76%,%nulo",532,0.384671,488,44,0.082707,-0.052372,0.001033,0.501679,"_44%,%_66%,%_52%,%_56%,%_67%,%_35%,%_55%,%_76%,%nulo",False
3,FM_category_septil,"_53%,%_54%,%_22%,%_42%,%_77%,%_24%,%_34%,%_64%,%_45%,%_36%,%_25%,%_75",496,0.358641,428,68,0.137097,0.514138,0.117116,0.501679,"_53%,%_54%,%_22%,%_42%,%_77%,%_24%,%_34%,%_64%,%_45%,%_36%,%_25%,%_75",False


In [49]:
data = add_new_var_x_cut_up(data, colum='PERCENT_GRUPO_ECON_VPC_SEGUIR', cortes=[0.1, 0.2], impute=0, up=True, soft=False)

>>  _0.1_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06355140186915888  PRECISION:  0.0787037037037037  F1:  0.0703205791106515  GINI:  0.018236589162794248
//////////////////////////////////////////////////
>>  _0.2_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.024299065420560748  PRECISION:  0.07027027027027027  F1:  0.036111111111111115  GINI:  0.004715779527358022
//////////////////////////////////////////////////


In [50]:
var = vars_top[2]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT_PREFERIDAS_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT_PREFERIDAS_SF,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.461575,missing,True
1,GAR_TOT_PREFERIDAS_SF,"[-inf,200000.0)",491,0.355025,465,26,0.052953,-0.530187,0.080326,0.461575,200000.0,False
2,GAR_TOT_PREFERIDAS_SF,"[200000.0,800000.0)",165,0.119306,146,19,0.115152,0.314586,0.013444,0.461575,800000.0,False
3,GAR_TOT_PREFERIDAS_SF,"[800000.0,1800000.0)",210,0.151844,202,8,0.038095,-0.875073,0.081618,0.461575,1800000.0,False
4,GAR_TOT_PREFERIDAS_SF,"[1800000.0,2600000.0)",88,0.063630,68,20,0.227273,1.129978,0.127492,0.461575,2600000.0,False
5,GAR_TOT_PREFERIDAS_SF,"[2600000.0,7000000.0)",234,0.169197,197,37,0.158120,0.681468,0.103825,0.461575,7000000.0,False
6,GAR_TOT_PREFERIDAS_SF,"[7000000.0,inf)",148,0.107014,139,9,0.060811,-0.383496,0.013444,0.461575,inf,False


In [51]:
texto_a_comparar = """
Transporte de personal%,%Alquiler de equipos%,%Estación de Servicio%,%Hotelería%,%Materiales de construcción y ferretería%,%Textil y calzado%,%Servicios diversos%,%Acondicionamiento de Edificios%,%Clínicas%,%Turismo%,%Taller Automotriz%,%Productos Eléctricos
""".upper()

otro_texto_adicional = """
Minería%,%Construcción de Edificios%,%Transporte interprovincial%,%Artículos diversos%,%Construcción de Carreteras%,%Publicidad%,%Bebidas alcohólicas%,%Maquinaria pesada y/o amarilla
""".upper()

data['cut_letal_SUBSECTOR_s2_provincia'] = data['SUBSECTOR'].apply(lambda _: None if pd.isnull(_) else (1 if _.upper() in texto_a_comparar.upper() else 0))
data['cut_deCuidado_SUBSECTOR'] = data['SUBSECTOR'].apply(
    lambda _: None if pd.isnull(_) else (1 if _.upper() in texto_a_comparar or _.upper() in otro_texto_adicional.upper() else 0))

In [52]:
var = vars_top[3]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COSTO_VENTAS_ANUAL_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COSTO_VENTAS_ANUAL_CEF_SITUACION,missing,331,0.239335,306,25,0.075529,-0.150956,0.005124,0.460036,missing,True
1,COSTO_VENTAS_ANUAL_CEF_SITUACION,"[-inf,4000.0)",440,0.318149,378,62,0.140909,0.545994,0.118688,0.460036,4000.0,False
2,COSTO_VENTAS_ANUAL_CEF_SITUACION,"[4000.0,11000.0)",337,0.243673,326,11,0.032641,-1.035249,0.172316,0.460036,11000.0,False
3,COSTO_VENTAS_ANUAL_CEF_SITUACION,"[11000.0,16000.0)",115,0.083153,99,16,0.139130,0.531222,0.029190,0.460036,16000.0,False
4,COSTO_VENTAS_ANUAL_CEF_SITUACION,"[16000.0,22000.0)",70,0.050615,65,5,0.071429,-0.211196,0.002069,0.460036,22000.0,False
5,COSTO_VENTAS_ANUAL_CEF_SITUACION,"[22000.0,inf)",90,0.065076,89,1,0.011111,-2.134883,0.132648,0.460036,inf,False


In [53]:
var = vars_top[4]
print(">>>>> ", var)
display(cortes[var])

>>>>>  AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,missing,38,0.027477,38,0,0.000000,-1.396665,0.031625,0.398534,missing,True
1,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,"[-inf,-600000.0)",109,0.078814,99,10,0.091743,0.053747,0.000233,0.398534,-600000.0,False
2,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,"[-600000.0,-360000.0)",84,0.060738,82,2,0.023810,-1.367291,0.066153,0.398534,-360000.0,False
3,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,"[-360000.0,-260000.0)",95,0.068691,85,10,0.105263,0.206215,0.003178,0.398534,-260000.0,False
4,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,"[-260000.0,-180000.0)",130,0.093999,104,26,0.200000,0.959987,0.127400,0.398534,-180000.0,False
5,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,"[-180000.0,-120000.0)",105,0.075922,100,5,0.047619,-0.649451,0.024562,0.398534,-120000.0,False
6,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,"[-120000.0,-80000.0)",101,0.073030,86,15,0.148515,0.599984,0.033586,0.398534,-80000.0,False
7,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,"[-80000.0,-40000.0)",196,0.141721,190,6,0.030612,-1.108983,0.111794,0.398534,-40000.0,False
8,AVG_TICKET_PROM_DEBT_DECREASE_U6M_ANIO_ACT,"[-40000.0,inf)",525,0.379610,479,46,0.087619,0.003222,0.000004,0.398534,inf,False


In [54]:
var = vars_top[5]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.395229,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1,"[-inf,1.0)",210,0.151844,165,45,0.214286,1.054470,0.257669,0.395229,1.0,False
2,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1,"[1.0,inf)",115,0.083153,101,14,0.121739,0.377690,0.013861,0.395229,inf,False


In [55]:
data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO_SF', cortes=[1,2,3,4,5], impute=0, up=True, soft=False)

>>  _1_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.22242990654205608  PRECISION:  0.17921686746987953  F1:  0.19849874895746455  GINI:  0.16037821577580313
//////////////////////////////////////////////////
>>  _2_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11121495327102804  PRECISION:  0.20101351351351351  F1:  0.14320096269554755  GINI:  0.08428793516787425
//////////////////////////////////////////////////
>>  _3_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08598130841121496  PRECISION:  0.22493887530562348  F1:  0.1244083840432725  GINI:  0.06793508274800186
//////////////////////////////////////////////////
>>  _4_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06542056074766354  PRECISION:  0.24305555555555555  F1:  0.10309278350515463  GINI:  0.053010222594412904
///////////////////////////////////////

In [56]:
var = vars_top[6]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.378201,missing,True
1,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[-inf,0.30000000000000004)",662,0.478670,590,72,0.108761,0.250297,0.033254,0.378201,0.30000000000000004,False
2,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[0.30000000000000004,0.7000000000000001)",253,0.182936,235,18,0.071146,-0.215460,0.007771,0.378201,0.7000000000000001,False
3,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[0.7000000000000001,1.0)",151,0.109183,128,23,0.152318,0.637217,0.057554,0.378201,1.0,False
4,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[1.0,inf)",221,0.159798,217,4,0.018100,-1.639850,0.227086,0.378201,inf,False


In [57]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_SIN_LINEA_VIGENTE', cortes=[1, 3, 5], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3037383177570093  PRECISION:  0.08807588075880758  F1:  0.13655462184873948  GINI:  0.1121750705749529
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.22429906542056074  PRECISION:  0.08456659619450317  F1:  0.12282497441146367  GINI:  0.07639971440154691
//////////////////////////////////////////////////
>>  _5_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1542056074766355  PRECISION:  0.07750117426021606  F1:  0.10315723663644889  GINI:  0.04239870778404775
//////////////////////////////////////////////////


In [58]:
display(cortes['DEPARTAMENTO'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEPARTAMENTO,"APURIMAC%,%PUNO%,%CUSCO%,%TUMBES%,%MADRE DE DIOS%,%ICA%,%CAJAMARCA",222,0.160521,220,2,0.009009,-2.346727,0.369661,0.605233,"APURIMAC%,%PUNO%,%CUSCO%,%TUMBES%,%MADRE DE DIOS%,%ICA%,%CAJAMARCA",False
1,DEPARTAMENTO,"LAMBAYEQUE%,%AREQUIPA%,%LIMA%,%TACNA",402,0.290672,379,23,0.057214,-0.448289,0.048600,0.605233,"LAMBAYEQUE%,%AREQUIPA%,%LIMA%,%TACNA",False
2,DEPARTAMENTO,"UCAYALI%,%JUNIN",177,0.127983,161,16,0.090395,0.044938,0.000263,0.605233,"UCAYALI%,%JUNIN",False
3,DEPARTAMENTO,"HUANUCO%,%LA LIBERTAD%,%AYACUCHO%,%SAN MARTIN",409,0.295734,364,45,0.110024,0.263262,0.022850,0.605233,"HUANUCO%,%LA LIBERTAD%,%AYACUCHO%,%SAN MARTIN",False
4,DEPARTAMENTO,"LORETO%,%ANCASH%,%PIURA%,%AMAZONAS%,%MOQUEGUA",173,0.125090,139,34,0.196532,0.945640,0.163859,0.605233,"LORETO%,%ANCASH%,%PIURA%,%AMAZONAS%,%MOQUEGUA",False


In [278]:
departamento_letal = 'LORETO%,%ANCASH%,%PIURA%,%AMAZONAS%,%MOQUEGUA'
departamento_preocupa = 'HUANUCO%,%LA LIBERTAD%,%AYACUCHO%,%SAN MARTIN'

data['departamento_letal'] = data['DEPARTAMENTO'].apply(lambda _: str(_).upper() in departamento_letal.upper()).astype(int)
data['departamento_preocupa'] = data['DEPARTAMENTO'].apply(
    lambda _: str(_).upper() in departamento_letal.upper() or str(_).upper() in departamento_preocupa.upper()).astype(int)

In [60]:
var = vars_top[7]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCETN_DEBT_IN__IBKSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCETN_DEBT_IN__IBKSF_ULT_RCC,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.378201,missing,True
1,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[-inf,0.30000000000000004)",662,0.478670,590,72,0.108761,0.250297,0.033254,0.378201,0.30000000000000004,False
2,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[0.30000000000000004,0.7000000000000001)",253,0.182936,235,18,0.071146,-0.215460,0.007771,0.378201,0.7000000000000001,False
3,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[0.7000000000000001,1.0)",151,0.109183,128,23,0.152318,0.637217,0.057554,0.378201,1.0,False
4,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[1.0,inf)",221,0.159798,217,4,0.018100,-1.639850,0.227086,0.378201,inf,False


In [61]:
var = vars_top[8]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_DOCS_LAB_U1M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_DOCS_LAB_U1M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.378155,missing,True
1,NRO_DOCS_LAB_U1M,"[-inf,2.0)",222,0.160521,177,45,0.202703,0.984266,0.231162,0.378155,2.0,False
2,NRO_DOCS_LAB_U1M,"[2.0,inf)",103,0.074476,89,14,0.135922,0.504174,0.023293,0.378155,inf,False


In [62]:
var = vars_top[9]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_DOCS_LAB_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_DOCS_LAB_U12M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.12370,0.37199,missing,True
1,NRO_DOCS_LAB_U12M,"[-inf,1.0)",193,0.139552,154,39,0.202073,0.980362,0.19908,0.37199,1.0,False
2,NRO_DOCS_LAB_U12M,"[1.0,inf)",132,0.095445,112,20,0.151515,0.630987,0.04921,0.37199,inf,False


In [63]:
var = vars_top[10]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.36864,missing,True
1,NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC,"[-inf,2.0)",637,0.460593,573,64,0.100471,0.161751,0.012884,0.36864,2.0,False
2,NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC,"[2.0,3.0)",318,0.229935,275,43,0.135220,0.498182,0.070043,0.36864,3.0,False
3,NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC,"[3.0,inf)",111,0.080260,105,6,0.054054,-0.508448,0.016848,0.36864,inf,False


In [64]:
data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO', cortes=[1,2,3,4,5], impute=0, up=True, soft=False)

>>  _1_cut_up_MAX_DIAS_ATRASO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1542056074766355  PRECISION:  0.1966626936829559  F1:  0.17286537454164483  GINI:  0.1158360298835579
//////////////////////////////////////////////////
>>  _2_cut_up_MAX_DIAS_ATRASO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11401869158878504  PRECISION:  0.21631205673758866  F1:  0.1493268053855569  GINI:  0.08885644634228629
//////////////////////////////////////////////////
>>  _3_cut_up_MAX_DIAS_ATRASO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09439252336448598  PRECISION:  0.2433734939759036  F1:  0.13602693602693602  GINI:  0.07651708217127173
//////////////////////////////////////////////////
>>  _4_cut_up_MAX_DIAS_ATRASO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06822429906542056  PRECISION:  0.2616487455197133  F1:  0.10822831727205337  GINI:  0.05649709879216536
//////////////////////////////////////////////////
>>  _5

In [65]:
data = add_new_var_x_cut_up(data, colum='NRO_FEVE_SEGUIR_GRUPO_EECON', cortes=[1,2, 5], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_FEVE_SEGUIR_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1542056074766355  PRECISION:  0.0746606334841629  F1:  0.10060975609756098  GINI:  0.0377875270940784
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_FEVE_SEGUIR_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.03644859813084112  PRECISION:  0.058823529411764705  F1:  0.04500865551067512  GINI:  0.0009254283710777855
//////////////////////////////////////////////////
>>  _5_cut_up_NUM_FEVE_SEGUIR_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0065420560747663555  PRECISION:  0.04046242774566474  F1:  0.011263073209975865  GINI:  -0.002908017931837259
//////////////////////////////////////////////////


In [66]:
var = vars_top[11]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_DOCS_LAB_U6M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_DOCS_LAB_U6M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.365134,missing,True
1,NRO_DOCS_LAB_U6M,"[-inf,4.0)",253,0.182936,209,44,0.173913,0.795609,0.160067,0.365134,4.0,False
2,NRO_DOCS_LAB_U6M,"[4.0,inf)",72,0.052061,57,15,0.208333,1.018752,0.081367,0.365134,inf,False


In [67]:
var = vars_top[12]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COD_GRUPO_ECONOMICO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COD_GRUPO_ECONOMICO,missing,87,0.062907,77,10,0.114943,0.312533,0.006991,0.36312,missing,True
1,COD_GRUPO_ECONOMICO,"[-inf,1900.0)",225,0.162690,213,12,0.053333,-0.522632,0.035877,0.36312,1900.0,False
2,COD_GRUPO_ECONOMICO,"[1900.0,2500.0)",77,0.055676,76,1,0.012987,-1.976980,0.102488,0.36312,2500.0,False
3,COD_GRUPO_ECONOMICO,"[2500.0,2900.0)",90,0.065076,79,11,0.122222,0.382201,0.011129,0.36312,2900.0,False
4,COD_GRUPO_ECONOMICO,"[2900.0,3900.0)",246,0.177874,229,17,0.069106,-0.246755,0.009783,0.36312,3900.0,False
5,COD_GRUPO_ECONOMICO,"[3900.0,4000.0)",82,0.059291,80,2,0.024390,-1.335126,0.062316,0.36312,4000.0,False
6,COD_GRUPO_ECONOMICO,"[4000.0,4500.0)",105,0.075922,85,20,0.190476,0.906834,0.090109,0.36312,4500.0,False
7,COD_GRUPO_ECONOMICO,"[4500.0,5000.0)",71,0.051338,68,3,0.042254,-0.767142,0.022124,0.36312,5000.0,False
8,COD_GRUPO_ECONOMICO,"[5000.0,inf)",400,0.289226,356,44,0.110000,0.263012,0.022303,0.36312,inf,False


In [68]:
data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x', cortes=[1,2, 5], impute=0, up=True, soft=False)

>>  _1_cut_up_MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1542056074766355  PRECISION:  0.1966626936829559  F1:  0.17286537454164483  GINI:  0.1158360298835579
//////////////////////////////////////////////////
>>  _2_cut_up_MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11401869158878504  PRECISION:  0.21631205673758866  F1:  0.1493268053855569  GINI:  0.08885644634228629
//////////////////////////////////////////////////
>>  _5_cut_up_MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.052336448598130844  PRECISION:  0.2978723404255319  F1:  0.0890302066772655  GINI:  0.044821931918180846
//////////////////////////////////////////////////


In [69]:
var = vars_top[13]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TENDENCIA_NRO_DOCS_LAB_U7M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TENDENCIA_NRO_DOCS_LAB_U7M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TENDENCIA_NRO_DOCS_LAB_U7M,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [70]:
var = vars_top[14]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [71]:
var = vars_top[15]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.1


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.1,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.1,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [72]:
data = add_new_var_x_cut_up(data, colum='COVER_GAR_AUTO_HIPO_ULT_RCC', cortes=[0.4, 0.6], impute=0, up=False, soft=True)

>>  _0.4_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6289719626168224  PRECISION:  0.052992125984251966  F1:  0.09774872912127815  GINI:  -0.05570297760861309
//////////////////////////////////////////////////
>>  _0.6_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7261682242990655  PRECISION:  0.05570291777188329  F1:  0.10346893934349824  GINI:  -0.023689455309268803
//////////////////////////////////////////////////


In [73]:
var = vars_top[16]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_DOCS_LAB_PROMEDIO_U6M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_DOCS_LAB_PROMEDIO_U6M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,NRO_DOCS_LAB_PROMEDIO_U6M,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [74]:
var = vars_top[17]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.9


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.9,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.9,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [75]:
data = add_new_var_x_cut_up(data, colum='COVER_GAR_AUTO_HIPO_ULT_RCC', cortes=[0.4, 0.6], impute=0, up=False, soft=True)

>>  _0.4_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6289719626168224  PRECISION:  0.052992125984251966  F1:  0.09774872912127815  GINI:  -0.05570297760861309
//////////////////////////////////////////////////
>>  _0.6_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7261682242990655  PRECISION:  0.05570291777188329  F1:  0.10346893934349824  GINI:  -0.023689455309268803
//////////////////////////////////////////////////


In [76]:
data = add_new_var_x_cut_up(data, colum='ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF', cortes=[1, 4, 5], impute=0, up=True, soft=False)

>>  _1_cut_up_ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.14205607476635515  PRECISION:  0.15154536390827517  F1:  0.14664737095996142  GINI:  0.09361021344334475
//////////////////////////////////////////////////
>>  _4_cut_up_ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09345794392523364  PRECISION:  0.234192037470726  F1:  0.1336005344021376  GINI:  0.07484243669535773
//////////////////////////////////////////////////
>>  _5_cut_up_ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08224299065420561  PRECISION:  0.2558139534883721  F1:  0.12446958981612449  GINI:  0.06766938254763621
//////////////////////////////////////////////////


In [77]:
var = vars_top[18]
print(">>>>> ", var)

display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.6


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.6,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.6,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [78]:
departamento_letal = 'Servicios%,%Pesca'
departamento_preocupa = 'Construcción%,%Comercio'

data['ACTIVIDAD_letal'] = data['ACTIVIDAD'].apply(lambda _: str(_).upper() in departamento_letal.upper()).astype(int)
data['ACTIVIDAD_preocupa'] = data['ACTIVIDAD'].apply(
    lambda _: str(_).upper() in departamento_letal.upper() or str(_).upper() in departamento_preocupa.upper()).astype(int)

In [79]:
var = vars_top[19]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.4


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.4,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.4,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [80]:
var = vars_top[20]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYOR_1


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYOR_1,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYOR_1,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [81]:
var = vars_top[21]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_TENDENCIA_CRECIENTE_NRO_DOCS_LAB_U7M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_TENDENCIA_CRECIENTE_NRO_DOCS_LAB_U7M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_TENDENCIA_CRECIENTE_NRO_DOCS_LAB_U7M,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [82]:
var = vars_top[22]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.7


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.7,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.7,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [83]:
data = add_new_var_x_cut_up(data, colum='ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF', cortes=[2, 4], impute=0, up=True, soft=False)

>>  _2_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.10093457943925234  PRECISION:  0.23580786026200873  F1:  0.14136125654450263  GINI:  0.08100972460605194
//////////////////////////////////////////////////
>>  _4_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0691588785046729  PRECISION:  0.26714801444043323  F1:  0.10987379361544172  GINI:  0.057602462701416446
//////////////////////////////////////////////////


In [84]:
var = vars_top[23]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.8


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.8,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.8,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [85]:
var = vars_top[24]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_TENDENCIA_CRECIENTE_NRO_DOCS_LAB_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_TENDENCIA_CRECIENTE_NRO_DOCS_LAB_U12M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_TENDENCIA_CRECIENTE_NRO_DOCS_LAB_U12M,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [86]:
var = vars_top[25]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TENDENCIA_NRO_DOCS_LAB_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TENDENCIA_NRO_DOCS_LAB_U12M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TENDENCIA_NRO_DOCS_LAB_U12M,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [87]:
var = vars_top[26]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_MESES_ANTERIORES_SIN_NRO_DOCS_LAB


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_MESES_ANTERIORES_SIN_NRO_DOCS_LAB,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,NRO_MESES_ANTERIORES_SIN_NRO_DOCS_LAB,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [88]:
var = vars_top[27]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_REGISTRADO_MES_ACTUAL


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_REGISTRADO_MES_ACTUAL,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_REGISTRADO_MES_ACTUAL,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [89]:
data = add_new_var_x_cut_up(data, colum='maximo_percent_col_direct_u12m', cortes=[0.66, 0.89], impute=0, up=True, soft=False)

>>  _0.66_cut_up_maximo_percent_col_direct_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.9046728971962616  PRECISION:  0.05860628443421929  F1:  0.11008131005856597  GINI:  0.019496989192162806
//////////////////////////////////////////////////
>>  _0.89_cut_up_maximo_percent_col_direct_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7934579439252336  PRECISION:  0.058888811819379897  F1:  0.1096403435139149  GINI:  0.021056714277049693
//////////////////////////////////////////////////


In [90]:
var = vars_top[28]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_DOCS_LAB_PROMEDIO_U9M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_DOCS_LAB_PROMEDIO_U9M,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,NRO_DOCS_LAB_PROMEDIO_U9M,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [91]:
var = vars_top[29]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [92]:
var = vars_top[30]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_MESES_ANTERIORES_CON_NRO_DOCS_LAB_CONSECUTIVOS


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_MESES_ANTERIORES_CON_NRO_DOCS_LAB_CONSECUTIVOS,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,NRO_MESES_ANTERIORES_CON_NRO_DOCS_LAB_CONSECUTIVOS,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [93]:
var = vars_top[31]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.5


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.5,missing,1058,0.765004,997,61,0.057656,-0.440124,0.123700,0.361978,missing,True
1,TIENE_NRO_DOCS_LAB_U1M_OVER_U12M_MAYORIGUAL_1.5,"[-inf,inf)",325,0.234996,266,59,0.181538,0.847795,0.238279,0.361978,inf,False


In [94]:
var = vars_top[32]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_OTRA_ENTIDAD_PRESTAMOS_Y_COMEX_VIGENTES_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_OTRA_ENTIDAD_PRESTAMOS_Y_COMEX_VIGENTES_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.361497,missing,True
1,TIENE_OTRA_ENTIDAD_PRESTAMOS_Y_COMEX_VIGENTES_SF_ULT_RCC,"[-inf,1.0)",987,0.713666,877,110,0.111449,0.277727,0.061735,0.361497,1.0,False
2,TIENE_OTRA_ENTIDAD_PRESTAMOS_Y_COMEX_VIGENTES_SF_ULT_RCC,"[1.0,inf)",79,0.057122,76,3,0.037975,-0.878368,0.030896,0.361497,inf,False


In [95]:
var = vars_top[33]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT_AUTO_HIPO_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT_AUTO_HIPO_SF,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.349471,missing,True
1,GAR_TOT_AUTO_HIPO_SF,"[-inf,200000.0)",590,0.426609,548,42,0.071186,-0.214852,0.018023,0.349471,200000.0,False
2,GAR_TOT_AUTO_HIPO_SF,"[200000.0,600000.0)",103,0.074476,89,14,0.135922,0.504174,0.023293,0.349471,600000.0,False
3,GAR_TOT_AUTO_HIPO_SF,"[600000.0,1200000.0)",146,0.105568,142,4,0.027397,-1.215779,0.096165,0.349471,1200000.0,False
4,GAR_TOT_AUTO_HIPO_SF,"[1200000.0,1800000.0)",95,0.068691,87,8,0.084211,-0.032713,0.000073,0.349471,1800000.0,False
5,GAR_TOT_AUTO_HIPO_SF,"[1800000.0,3000000.0)",107,0.077368,84,23,0.214953,1.058431,0.132471,0.349471,3000000.0,False
6,GAR_TOT_AUTO_HIPO_SF,"[3000000.0,6600000.0)",159,0.114967,139,20,0.125786,0.415012,0.023494,0.349471,6600000.0,False
7,GAR_TOT_AUTO_HIPO_SF,"[6600000.0,inf)",136,0.098337,128,8,0.058824,-0.418835,0.014525,0.349471,inf,False


In [96]:
var = vars_top[34]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERIODO_ZONAL


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERIODO_ZONAL,"202101_BEP ZONAL 3%,%202102_BEP ZONAL 3",134,0.096891,132,2,0.014925,-1.835901,0.161277,0.338615,"202101_BEP ZONAL 3%,%202102_BEP ZONAL 3",False
1,PERIODO_ZONAL,"202012_BEP ZONAL 3%,%202011_BEP ZONAL 1%,%202101_BEP ZONAL 2",221,0.159798,211,10,0.045249,-0.695520,0.058235,0.338615,"202012_BEP ZONAL 3%,%202011_BEP ZONAL 1%,%202101_BEP ZONAL 2",False
2,PERIODO_ZONAL,"202011_BEP ZONAL 2%,%202009_BEP ZONAL 3%,%202010_BEP ZONAL 2%,%202102_BEP ZONAL 2%,%202012_BEP ZONAL 2%,%202010_BEP ZONAL 1%,%202012_BEP ZONAL 1%,%202102_BEP ZONAL 1",638,0.461316,584,54,0.084639,-0.027164,0.000337,0.338615,"202011_BEP ZONAL 2%,%202009_BEP ZONAL 3%,%202010_BEP ZONAL 2%,%202102_BEP ZONAL 2%,%202012_BEP ZONAL 2%,%202010_BEP ZONAL 1%,%202012_BEP ZONAL 1%,%202102_BEP ZONAL 1",False
3,PERIODO_ZONAL,"202011_BEP ZONAL 3%,%202010_BEP ZONAL 3%,%202009_BEP ZONAL 2%,%202101_BEP ZONAL 1",297,0.214751,261,36,0.121212,0.372752,0.034796,0.338615,"202011_BEP ZONAL 3%,%202010_BEP ZONAL 3%,%202009_BEP ZONAL 2%,%202101_BEP ZONAL 1",False
4,PERIODO_ZONAL,202009_BEP ZONAL 1,93,0.067245,75,18,0.193548,0.926637,0.083970,0.338615,202009_BEP ZONAL 1,False


In [97]:
var = vars_top[35]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_COMEX_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_COMEX_VIGENTE_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.332012,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_COMEX_VIGENTE_SF_ULT_RCC,"[-inf,1.0)",982,0.710051,874,108,0.109980,0.262804,0.054662,0.332012,1.0,False
2,TIENE_EN_OTRAENTIDAD_DEBT_COMEX_VIGENTE_SF_ULT_RCC,"[1.0,inf)",84,0.060738,79,5,0.059524,-0.406257,0.008484,0.332012,inf,False


In [98]:
var = vars_top[36]
print(">>>>> ", var)
display(cortes[var])

>>>>>  maximo_percent_sow_ibk_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,maximo_percent_sow_ibk_u12m,missing,25,0.018077,24,1,0.040000,-0.824300,0.008794,0.327624,missing,True
1,maximo_percent_sow_ibk_u12m,"[-inf,0.5)",649,0.469270,579,70,0.107858,0.240946,0.030094,0.327624,0.5,False
2,maximo_percent_sow_ibk_u12m,"[0.5,0.7000000000000001)",113,0.081706,106,7,0.061947,-0.363776,0.009310,0.327624,0.7000000000000001,False
3,maximo_percent_sow_ibk_u12m,"[0.7000000000000001,1.0)",193,0.139552,164,29,0.150259,0.621183,0.069459,0.327624,1.0,False
4,maximo_percent_sow_ibk_u12m,"[1.0,inf)",403,0.291396,390,13,0.032258,-1.047444,0.209966,0.327624,inf,False


In [99]:
var = vars_top[37]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_U6M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_U6M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.32431,missing,True
1,IMPORTE_LAB_U6M,"[-inf,1450.0)",308,0.222704,265,43,0.139610,0.535224,0.079489,0.32431,1450.0,False
2,IMPORTE_LAB_U6M,"[1450.0,inf)",79,0.057122,60,19,0.240506,1.203848,0.133419,0.32431,inf,False


In [100]:
var = vars_top[38]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.322001,missing,True
1,GAR_TOT,"[-inf,1000000.0)",335,0.242227,312,23,0.068657,-0.253756,0.014049,0.322001,1000000.0,False
2,GAR_TOT,"[1000000.0,2000000.0)",192,0.138829,185,7,0.036458,-0.920692,0.081153,0.322001,2000000.0,False
3,GAR_TOT,"[2000000.0,4000000.0)",189,0.136659,160,29,0.153439,0.645875,0.074265,0.322001,4000000.0,False
4,GAR_TOT,"[4000000.0,9000000.0)",244,0.176428,220,24,0.098361,0.138180,0.003567,0.322001,9000000.0,False
5,GAR_TOT,"[9000000.0,12000000.0)",105,0.075922,92,13,0.123810,0.396914,0.014087,0.322001,12000000.0,False
6,GAR_TOT,"[12000000.0,16000000.0)",98,0.070860,94,4,0.040816,-0.803247,0.033008,0.322001,16000000.0,False
7,GAR_TOT,"[16000000.0,29000000.0)",93,0.067245,78,15,0.161290,0.705095,0.044592,0.322001,29000000.0,False
8,GAR_TOT,"[29000000.0,inf)",80,0.057845,76,4,0.050000,-0.590686,0.015855,0.322001,inf,False


In [101]:
var = vars_top[39]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VIGENTE_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.320096,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VIGENTE_SF_ULT_RCC,"[-inf,1.0)",84,0.060738,72,12,0.142857,0.561994,0.024162,0.320096,1.0,False
2,TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VIGENTE_SF_ULT_RCC,"[1.0,inf)",982,0.710051,881,101,0.102851,0.187816,0.027068,0.320096,inf,False


In [102]:
data = add_new_var_x_cut_up(data, colum='PERCENT_GAR_OTROS_BANCOS_PRIN', cortes=[0.4], impute=0, up=True, soft=False)

>>  _0.4_cut_up_PERCENT_GAR_OTROS_BANCOS_PRIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6299065420560748  PRECISION:  0.06893025158519124  F1:  0.12426253687315633  GINI:  0.11163260376619655
//////////////////////////////////////////////////


In [103]:
var = vars_top[40]
print(">>>>> ", var)
display(cortes[var])

>>>>>  SOW_IBK_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SOW_IBK_ULT_RCC,missing,25,0.018077,24,1,0.040000,-0.824300,0.008794,0.319873,missing,True
1,SOW_IBK_ULT_RCC,"[-inf,0.1)",457,0.330441,420,37,0.080963,-0.075583,0.001830,0.319873,0.1,False
2,SOW_IBK_ULT_RCC,"[0.1,0.30000000000000004)",222,0.160521,189,33,0.148649,0.608514,0.076281,0.319873,0.30000000000000004,False
3,SOW_IBK_ULT_RCC,"[0.30000000000000004,1.0)",435,0.314534,391,44,0.101149,0.169235,0.009661,0.319873,1.0,False
4,SOW_IBK_ULT_RCC,"[1.0,inf)",244,0.176428,239,5,0.020492,-1.513272,0.223307,0.319873,inf,False


In [279]:
data = add_new_var_x_cut_up(data, colum='SOW_IBK_ULT_RCC', cortes=[0.4, 0.3, 0.2, 0.1, 0], impute=0, up=False, soft=False)

>>  _0.4_cut_down_SOW_IBK_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6065420560747663  PRECISION:  0.06897651185035604  F1:  0.12386678118141044  GINI:  0.10785140367809087
//////////////////////////////////////////////////
>>  _0.3_cut_down_SOW_IBK_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5364485981308411  PRECISION:  0.07037763609612555  F1:  0.12443095599393018  GINI:  0.10481931428705193
//////////////////////////////////////////////////
>>  _0.2_cut_down_SOW_IBK_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4289719626168224  PRECISION:  0.06870229007633588  F1:  0.11843633079602633  GINI:  0.07476497183918385
//////////////////////////////////////////////////
>>  _0.1_cut_down_SOW_IBK_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.28130841121495326  PRECISION:  0.059805285118219746  F1:  0.09864001310830738  GINI:  0.011924373870082494
////////////////////////////////////

In [104]:
data = add_new_var_x_cut_up(data, colum='minimo_percent_auto_hipo_u12m', cortes=[0.4], impute=0, up=False, soft=True)

>>  _0.4_cut_down_minimo_percent_auto_hipo_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7906542056074767  PRECISION:  0.06010657193605684  F1:  0.11172003961703533  GINI:  0.03755162106916399
//////////////////////////////////////////////////


In [105]:
var = vars_top[41]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COSTO_VENTAS_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COSTO_VENTAS_CEF_SITUACION,missing,331,0.239335,306,25,0.075529,-0.150956,0.005124,0.317934,missing,True
1,COSTO_VENTAS_CEF_SITUACION,"[-inf,2000.0)",343,0.248012,293,50,0.145773,0.585604,0.108149,0.317934,2000.0,False
2,COSTO_VENTAS_CEF_SITUACION,"[2000.0,3000.0)",130,0.093999,117,13,0.100000,0.156529,0.002457,0.317934,3000.0,False
3,COSTO_VENTAS_CEF_SITUACION,"[3000.0,8000.0)",311,0.224873,297,14,0.045016,-0.700921,0.083051,0.317934,8000.0,False
4,COSTO_VENTAS_CEF_SITUACION,"[8000.0,21000.0)",184,0.133044,167,17,0.092391,0.068973,0.000651,0.317934,21000.0,False
5,COSTO_VENTAS_CEF_SITUACION,"[21000.0,inf)",84,0.060738,83,1,0.011905,-2.065087,0.118501,0.317934,inf,False


In [106]:
var = vars_top[42]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE__OTRA_ENTIDAD_PRESTAMOS__Y_LEASING_VIGENTES_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE__OTRA_ENTIDAD_PRESTAMOS__Y_LEASING_VIGENTES_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.317719,missing,True
1,TIENE__OTRA_ENTIDAD_PRESTAMOS__Y_LEASING_VIGENTES_SF_ULT_RCC,"[-inf,1.0)",703,0.508315,633,70,0.099573,0.151778,0.012468,0.317719,1.0,False
2,TIENE__OTRA_ENTIDAD_PRESTAMOS__Y_LEASING_VIGENTES_SF_ULT_RCC,"[1.0,inf)",363,0.262473,320,43,0.118457,0.346632,0.036385,0.317719,inf,False


In [107]:
var = vars_top[43]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_LEASING_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_LEASING_VIGENTE_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.31605,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_LEASING_VIGENTE_SF_ULT_RCC,"[-inf,1.0)",633,0.457701,570,63,0.099526,0.151252,0.011146,0.31605,1.0,False
2,TIENE_EN_OTRAENTIDAD_DEBT_LEASING_VIGENTE_SF_ULT_RCC,"[1.0,inf)",433,0.313087,383,50,0.115473,0.317741,0.036038,0.31605,inf,False


In [108]:
var = vars_top[44]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VIGENTE_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.314758,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VIGENTE_SF_ULT_RCC,"[-inf,1.0)",973,0.703543,872,101,0.103803,0.198084,0.029960,0.314758,1.0,False
2,TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VIGENTE_SF_ULT_RCC,"[1.0,inf)",93,0.067245,81,12,0.129032,0.444211,0.015933,0.314758,inf,False


In [109]:
data = add_new_var_x_cut_up(data, colum='PERCENT_GAR_OTROS_BANCOS_PRIN', cortes=[0.2, 0.4, 0.5], impute=0, up=True, soft=False)

>>  _0.2_cut_up_PERCENT_GAR_OTROS_BANCOS_PRIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7093457943925233  PRECISION:  0.06854510972636142  F1:  0.12501029399654123  GINI:  0.12218878653643772
//////////////////////////////////////////////////
>>  _0.4_cut_up_PERCENT_GAR_OTROS_BANCOS_PRIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6299065420560748  PRECISION:  0.06893025158519124  F1:  0.12426253687315633  GINI:  0.11163260376619655
//////////////////////////////////////////////////
>>  _0.5_cut_up_PERCENT_GAR_OTROS_BANCOS_PRIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5766355140186916  PRECISION:  0.06759421560035057  F1:  0.12100411845459895  GINI:  0.09211997263192173
//////////////////////////////////////////////////


In [110]:
var = vars_top[45]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_OTROS_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_OTROS_VIGENTE_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_OTROS_VIGENTE_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [111]:
data = add_new_var_x_cut_up(data, colum='minimo_percent_auto_hipo_u12m', cortes=[0.4, 0.3,0.2, 0.1], impute=0, up=False, soft=True)

>>  _0.4_cut_down_minimo_percent_auto_hipo_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7906542056074767  PRECISION:  0.06010657193605684  F1:  0.11172003961703533  GINI:  0.03755162106916399
//////////////////////////////////////////////////
>>  _0.3_cut_down_minimo_percent_auto_hipo_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.708411214953271  PRECISION:  0.05777439024390244  F1:  0.10683579985905568  GINI:  0.004665342244629445
//////////////////////////////////////////////////
>>  _0.2_cut_down_minimo_percent_auto_hipo_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.625233644859813  PRECISION:  0.05564335024536306  F1:  0.10219201099824335  GINI:  -0.021128645929211265
//////////////////////////////////////////////////
>>  _0.1_cut_down_minimo_percent_auto_hipo_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5186915887850467  PRECISION:  0.05184977578475336  F1:  0.09427552233735349  GINI:  -0

In [112]:
var = vars_top[46]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TOTAL


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TOTAL,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TOTAL,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [113]:
var = vars_top[47]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VENCIDO_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VENCIDO_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VENCIDO_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [114]:
var = vars_top[48]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_SOBREGIROS__AVANCES_VENCIDO_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_SOBREGIROS__AVANCES_VENCIDO_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_SOBREGIROS__AVANCES_VENCIDO_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [115]:
var = vars_top[49]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_FACTORING_VENCIDO_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_FACTORING_VENCIDO_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_FACTORING_VENCIDO_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [116]:
var = vars_top[50]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_SOBREGIROS__AVANCES_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_SOBREGIROS__AVANCES_VIGENTE_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_SOBREGIROS__AVANCES_VIGENTE_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [117]:
var = vars_top[51]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VENCIDO_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VENCIDO_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VENCIDO_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [118]:
var = vars_top[52]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_PROD_OTRA_ENTIDAD_DEUDAS_VENCIDO_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_PROD_OTRA_ENTIDAD_DEUDAS_VENCIDO_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,NRO_PROD_OTRA_ENTIDAD_DEUDAS_VENCIDO_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [119]:
var = vars_top[53]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_FACTORING_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_FACTORING_VIGENTE_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_FACTORING_VIGENTE_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [120]:
var = vars_top[54]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE__OTRA_ENTIDAD_PRESTAMOS_S_COMEX_Y_LEASING_VIGENTES_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE__OTRA_ENTIDAD_PRESTAMOS_S_COMEX_Y_LEASING_VIGENTES_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE__OTRA_ENTIDAD_PRESTAMOS_S_COMEX_Y_LEASING_VIGENTES_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [121]:
var = vars_top[55]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_LEASING_VENCIDO_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_LEASING_VENCIDO_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_LEASING_VENCIDO_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [122]:
var = vars_top[56]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_EN_OTRAENTIDAD_DEBT_COMEX_VENCIDO_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_EN_OTRAENTIDAD_DEBT_COMEX_VENCIDO_SF_ULT_RCC,missing,317,0.229212,310,7,0.022082,-1.436909,0.268866,0.310316,missing,True
1,TIENE_EN_OTRAENTIDAD_DEBT_COMEX_VENCIDO_SF_ULT_RCC,"[-inf,inf)",1066,0.770788,953,113,0.106004,0.221526,0.041451,0.310316,inf,False


In [123]:
data = add_new_var_x_cut_up(data, colum='minimo_percent_others_u12m', cortes=[0.4, 0.5], impute=0, up=True, soft=False)

>>  _0.4_cut_up_minimo_percent_others_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5747663551401869  PRECISION:  0.06991019665795158  F1:  0.12465795074490725  GINI:  0.10898017729662546
//////////////////////////////////////////////////
>>  _0.5_cut_up_minimo_percent_others_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5205607476635514  PRECISION:  0.06904673360604933  F1:  0.12192185618912116  GINI:  0.09303029109973493
//////////////////////////////////////////////////


In [124]:
var = vars_top[57]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_SFSF_ULT_RCC_x


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_SFSF_ULT_RCC_x,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.305608,missing,True
1,MAX_DIAS_ATRASO_SFSF_ULT_RCC_x,"[-inf,1.0)",1202,0.869125,1110,92,0.076539,-0.136573,0.015323,0.305608,1.0,False
2,MAX_DIAS_ATRASO_SFSF_ULT_RCC_x,"[1.0,inf)",85,0.061461,60,25,0.294118,1.478285,0.237749,0.305608,inf,False


In [125]:
var = vars_top[58]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_U12M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.304178,missing,True
1,IMPORTE_LAB_U12M,"[-inf,1050.0)",305,0.220535,260,45,0.147541,0.599734,0.101440,0.304178,1050.0,False
2,IMPORTE_LAB_U12M,"[1050.0,inf)",82,0.059291,65,17,0.207317,1.012579,0.091337,0.304178,inf,False


In [126]:
var = vars_top[59]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_PREN_NO_PREFER


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_PREN_NO_PREFER,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.303996,missing,True
1,GAR_PREN_NO_PREFER,"[-inf,500000.0)",381,0.275488,348,33,0.086614,-0.001942,0.000001,0.303996,500000.0,False
2,GAR_PREN_NO_PREFER,"[500000.0,2000000.0)",300,0.216920,282,18,0.060000,-0.397782,0.029149,0.303996,2000000.0,False
3,GAR_PREN_NO_PREFER,"[2000000.0,2500000.0)",72,0.052061,53,19,0.263889,1.327900,0.154527,0.303996,2500000.0,False
4,GAR_PREN_NO_PREFER,"[2500000.0,6000000.0)",254,0.183659,238,16,0.062992,-0.345929,0.019063,0.303996,6000000.0,False
5,GAR_PREN_NO_PREFER,"[6000000.0,15000000.0)",209,0.151121,182,27,0.129187,0.445584,0.036047,0.303996,15000000.0,False
6,GAR_PREN_NO_PREFER,"[15000000.0,inf)",120,0.086768,114,6,0.050000,-0.590686,0.023782,0.303996,inf,False


In [127]:
data = add_new_var_x_cut_up(data, colum='minimo_percent_auto_hipo_pren_u12m', cortes=[0.4], impute=0, up=False, soft=True)

>>  _0.4_cut_down_minimo_percent_auto_hipo_pren_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7383177570093458  PRECISION:  0.05917159763313609  F1:  0.1095624436585535  GINI:  0.023243181124112944
//////////////////////////////////////////////////


In [128]:
var = vars_top[60]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29686,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1,"[-inf,1.0)",271,0.195951,225,46,0.169742,0.766294,0.157233,0.29686,1.0,False
2,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1,"[1.0,inf)",116,0.083876,100,16,0.137931,0.521172,0.028225,0.29686,inf,False


In [129]:
var = vars_top[61]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_REGISTRADO_MAYOR_A_2.5MIL_SOLES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_REGISTRADO_MAYOR_A_2.5MIL_SOLES,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.292784,missing,True
1,IMPORTE_LAB_REGISTRADO_MAYOR_A_2.5MIL_SOLES,"[-inf,1.0)",311,0.224873,262,49,0.157556,0.677229,0.136049,0.292784,1.0,False
2,IMPORTE_LAB_REGISTRADO_MAYOR_A_2.5MIL_SOLES,"[1.0,inf)",76,0.054953,63,13,0.171053,0.775568,0.045334,0.292784,inf,False


In [130]:
var = vars_top[62]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_U1M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_U1M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.292612,missing,True
1,IMPORTE_LAB_U1M,"[-inf,600.0)",276,0.199566,231,45,0.163043,0.717998,0.137929,0.292612,600.0,False
2,IMPORTE_LAB_U1M,"[600.0,inf)",111,0.080260,94,17,0.153153,0.643672,0.043281,0.292612,inf,False


In [131]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_SIN_LINEA_VIGENTE', cortes=[1, 3], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3037383177570093  PRECISION:  0.08807588075880758  F1:  0.13655462184873948  GINI:  0.1121750705749529
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.22429906542056074  PRECISION:  0.08456659619450317  F1:  0.12282497441146367  GINI:  0.07639971440154691
//////////////////////////////////////////////////


In [132]:
var = vars_top[63]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_REGISTRADO_MAYOR_A_MIL_SOLES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_REGISTRADO_MAYOR_A_MIL_SOLES,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,IMPORTE_LAB_REGISTRADO_MAYOR_A_MIL_SOLES,"[-inf,1.0)",287,0.207520,241,46,0.160279,0.697598,0.134300,0.29218,1.0,False
2,IMPORTE_LAB_REGISTRADO_MAYOR_A_MIL_SOLES,"[1.0,inf)",100,0.072307,84,16,0.160000,0.695525,0.046478,0.29218,inf,False


In [133]:
var = vars_top[64]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_REGISTRADO_MAYOR_A_5MIL_SOLES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_REGISTRADO_MAYOR_A_5MIL_SOLES,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,IMPORTE_LAB_REGISTRADO_MAYOR_A_5MIL_SOLES,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [134]:
data = add_new_var_x_cut_up(data, colum='ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF', cortes=[2, 3, 4], impute=0, up=True, soft=False)

>>  _2_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.10093457943925234  PRECISION:  0.23580786026200873  F1:  0.14136125654450263  GINI:  0.08100972460605194
//////////////////////////////////////////////////
>>  _3_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08317757009345794  PRECISION:  0.239247311827957  F1:  0.12343966712898752  GINI:  0.06706690175689856
//////////////////////////////////////////////////
>>  _4_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0691588785046729  PRECISION:  0.26714801444043323  F1:  0.10987379361544172  GINI:  0.057602462701416446
//////////////////////////////////////////////////


In [135]:
var = vars_top[65]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_REGISTRADO_MAYOR_A_20MIL_SOLES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_REGISTRADO_MAYOR_A_20MIL_SOLES,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,IMPORTE_LAB_REGISTRADO_MAYOR_A_20MIL_SOLES,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [136]:
data = add_new_var_x_cut_up(data, colum='ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF', cortes=[2, 3, 4], impute=0, up=True, soft=False)

>>  _2_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.10093457943925234  PRECISION:  0.23580786026200873  F1:  0.14136125654450263  GINI:  0.08100972460605194
//////////////////////////////////////////////////
>>  _3_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08317757009345794  PRECISION:  0.239247311827957  F1:  0.12343966712898752  GINI:  0.06706690175689856
//////////////////////////////////////////////////
>>  _4_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0691588785046729  PRECISION:  0.26714801444043323  F1:  0.10987379361544172  GINI:  0.057602462701416446
//////////////////////////////////////////////////


In [137]:
var = vars_top[66]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_PROMEDIO_U6M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_PROMEDIO_U6M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,IMPORTE_LAB_PROMEDIO_U6M,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [138]:
data = add_new_var_x_cut_up(data, colum='BUCKET', cortes=[2, 3, 4], impute=0, up=True, soft=False)

>>  _2_cut_up_BUCKET
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4392523364485981  PRECISION:  0.0702646135446255  F1:  0.12114963268462431  GINI:  0.08521613014095841
//////////////////////////////////////////////////
>>  _3_cut_up_BUCKET
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.21495327102803738  PRECISION:  0.07271577616187164  F1:  0.10866997401370185  GINI:  0.04798298752581709
//////////////////////////////////////////////////
>>  _4_cut_up_BUCKET
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.18130841121495328  PRECISION:  0.07695358984529949  F1:  0.10804789752158174  GINI:  0.04883659065250301
//////////////////////////////////////////////////


In [139]:
var = vars_top[67]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.2,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [140]:
var = vars_top[68]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYOR_1


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYOR_1,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYOR_1,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [141]:
var = vars_top[69]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.5


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.5,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.5,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [142]:
var = vars_top[70]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.4


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.4,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.4,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [143]:
var = vars_top[71]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_TENDENCIA_CRECIENTE_IMPORTE_LAB_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_TENDENCIA_CRECIENTE_IMPORTE_LAB_U12M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_TENDENCIA_CRECIENTE_IMPORTE_LAB_U12M,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [144]:
var = vars_top[72]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_REGISTRADO_MAYOR_A_10MIL_SOLES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_REGISTRADO_MAYOR_A_10MIL_SOLES,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,IMPORTE_LAB_REGISTRADO_MAYOR_A_10MIL_SOLES,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [145]:
var = vars_top[73]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_TENDENCIA_CRECIENTE_IMPORTE_LAB_U7M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_TENDENCIA_CRECIENTE_IMPORTE_LAB_U7M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_TENDENCIA_CRECIENTE_IMPORTE_LAB_U7M,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [146]:
data = add_new_var_x_cut_up(data, colum='VARIACION_PERCENT_GAR_OTROS_BANCOS_PRIN_U12M', cortes=[0], impute=0, up=False, soft=True)

>>  _0_cut_down_VARIACION_PERCENT_GAR_OTROS_BANCOS_PRIN_U12M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.14579439252336449  PRECISION:  0.06011560693641618  F1:  0.08512960436562073  GINI:  0.006946618414290118
//////////////////////////////////////////////////


In [147]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M', cortes=[1,2], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3401869158878505  PRECISION:  0.07172413793103448  F1:  0.11847030105777054  GINI:  0.07199836983297181
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.2457943925233645  PRECISION:  0.0763203714451538  F1:  0.11647475642161205  GINI:  0.06459206985457255
//////////////////////////////////////////////////


In [148]:
var = vars_top[74]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_MESES_ANTERIORES_CON_IMPORTE_LAB_CONSECUTIVOS


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_MESES_ANTERIORES_CON_IMPORTE_LAB_CONSECUTIVOS,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,NRO_MESES_ANTERIORES_CON_IMPORTE_LAB_CONSECUTIVOS,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [149]:
data = add_new_var_x_cut_up(data, colum='minimo_percent_sow_others_u12m', cortes=[0.6], impute=0, up=True, soft=False)

>>  _0.6_cut_up_minimo_percent_sow_others_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.35887850467289717  PRECISION:  0.06277587052476705  F1:  0.10685960762487824  GINI:  0.03250938250507285
//////////////////////////////////////////////////


In [150]:
var = vars_top[75]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TENDENCIA_IMPORTE_LAB_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TENDENCIA_IMPORTE_LAB_U12M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TENDENCIA_IMPORTE_LAB_U12M,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [151]:
display(cortes['NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF,"[-inf,inf)",1383,1.0,1263,120,0.086768,0.0,0.0,0.0,inf,False


In [152]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF', cortes=[1, 2,5,6], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1691588785046729  PRECISION:  0.14811783960720132  F1:  0.1579406631762653  GINI:  0.1098966674150681
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.12990654205607477  PRECISION:  0.17866323907455012  F1:  0.15043290043290042  GINI:  0.09352944994631729
//////////////////////////////////////////////////
>>  _5_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.048598130841121495  PRECISION:  0.29714285714285715  F1:  0.08353413654618473  GINI:  0.041595967571168124
//////////////////////////////////////////////////
>>  _6_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.016822429906542057  PRECISION:  0.2857142857142857  F1:  0.0317740511

In [153]:
var = vars_top[76]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.8


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.8,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.8,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [154]:
var = vars_top[77]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_MESES_ANTERIORES_SIN_IMPORTE_LAB


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_MESES_ANTERIORES_SIN_IMPORTE_LAB,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,NRO_MESES_ANTERIORES_SIN_IMPORTE_LAB,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [155]:
var = vars_top[78]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.7


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.7,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.7,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [156]:
var = vars_top[79]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_REGISTRADO_MAYOR_A_7.5MIL_SOLES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_REGISTRADO_MAYOR_A_7.5MIL_SOLES,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,IMPORTE_LAB_REGISTRADO_MAYOR_A_7.5MIL_SOLES,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [157]:
data = add_new_var_x_cut_up(data, colum='_int_FM_category_quartil', cortes=[9, 12], impute=0, up=True, soft=False)

>>  _9_cut_up__int_FM_category_quartil
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.38037383177570094  PRECISION:  0.061258278145695365  F1:  0.10552242675654655  GINI:  0.02531291864806784
//////////////////////////////////////////////////
>>  _12_cut_up__int_FM_category_quartil
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.26822429906542056  PRECISION:  0.0591996699669967  F1:  0.09699222710375126  GINI:  0.008574976510484777
//////////////////////////////////////////////////


In [158]:
var = vars_top[80]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.1


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.1,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.1,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [159]:
var = vars_top[81]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_REGISTRADO_MES_ACTUAL


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_REGISTRADO_MES_ACTUAL,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_REGISTRADO_MES_ACTUAL,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [160]:
var = vars_top[82]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_REGISTRADO_MAYOR_A_15MIL_SOLES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_REGISTRADO_MAYOR_A_15MIL_SOLES,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,IMPORTE_LAB_REGISTRADO_MAYOR_A_15MIL_SOLES,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [161]:
var = vars_top[83]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.6


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.6,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.6,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [162]:
data = add_new_var_x_cut_up(data, colum='NRO_DOCS_TRIB_U1M', cortes=[1], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_DOCS_TRIB_U1M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.07009345794392523  PRECISION:  0.09689922480620156  F1:  0.08134490238611713  GINI:  0.030300676434190432
//////////////////////////////////////////////////


In [163]:
for col in [_ for _ in data.columns if  _.startswith('NRO_DOCS')]:
    data = add_new_var_x_cut_up(data, colum=col, cortes=[1, 2, 3], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_DOCS_TRIB_U1M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.07009345794392523  PRECISION:  0.09689922480620156  F1:  0.08134490238611713  GINI:  0.030300676434190432
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_DOCS_TRIB_U1M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04205607476635514  PRECISION:  0.1282051282051282  F1:  0.0633356790992259  GINI:  0.024636058826471352
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_DOCS_TRIB_U1M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.027102803738317756  PRECISION:  0.145  F1:  0.04566929133858268  GINI:  0.01736808894838271
//////////////////////////////////////////////////
>>  _1_cut_up_NUM_DOCS_TRIB_U6M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06635514018691589  PRECISION:  0.13003663003663005  F1:  0.08787128712871288  GINI:  0.03931426577042951
//////////////////////////////////////////////////
>>  _

In [164]:
data = add_new_var_x_cut_up(data, colum='_int_FM_category_quartil', cortes=[9, 12], impute=0, up=True, soft=False)

>>  _9_cut_up__int_FM_category_quartil
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.38037383177570094  PRECISION:  0.061258278145695365  F1:  0.10552242675654655  GINI:  0.02531291864806784
//////////////////////////////////////////////////
>>  _12_cut_up__int_FM_category_quartil
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.26822429906542056  PRECISION:  0.0591996699669967  F1:  0.09699222710375126  GINI:  0.008574976510484777
//////////////////////////////////////////////////


In [165]:
var = vars_top[84]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.9


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.9,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.9,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [166]:
data = add_new_var_x_cut_up(data, colum='MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF', cortes=[3000000.0, 5500000.0], impute=0, up=True, soft=False)

>>  _3000000.0_cut_up_MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6056074766355141  PRECISION:  0.06522395571212884  F1:  0.11776465243071332  GINI:  0.07691568567570561
//////////////////////////////////////////////////
>>  _5500000.0_cut_up_MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4672897196261682  PRECISION:  0.06770480704129993  F1:  0.11827321111768184  GINI:  0.07533936097878136
//////////////////////////////////////////////////


In [167]:
display(cortes['SOW_OTROSBCOS_MAX_historia'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SOW_OTROSBCOS_MAX_historia,missing,17,0.012292,17,0,0.000000,-0.592292,0.003563,0.102223,missing,True
1,SOW_OTROSBCOS_MAX_historia,"[-inf,0.1)",330,0.238612,310,20,0.060606,-0.394559,0.031573,0.102223,0.1,False
2,SOW_OTROSBCOS_MAX_historia,"[0.1,0.30000000000000004)",88,0.063630,81,7,0.079545,-0.102258,0.000637,0.102223,0.30000000000000004,False
3,SOW_OTROSBCOS_MAX_historia,"[0.30000000000000004,0.7000000000000001)",250,0.180766,218,32,0.128000,0.427522,0.039365,0.102223,0.7000000000000001,False
4,SOW_OTROSBCOS_MAX_historia,"[0.7000000000000001,0.9)",220,0.159074,198,22,0.100000,0.149057,0.003756,0.102223,0.9,False
5,SOW_OTROSBCOS_MAX_historia,"[0.9,1.0)",288,0.208243,269,19,0.065972,-0.303991,0.016972,0.102223,1.0,False
6,SOW_OTROSBCOS_MAX_historia,"[1.0,inf)",190,0.137383,170,20,0.105263,0.206215,0.006357,0.102223,inf,False


In [168]:
data = add_new_var_x_cut_up(data, colum='SOW_OTROSBCOS_MAX_historia', cortes=[0.5, 0.3], impute=0, up=False, soft=True)

>>  _0.5_cut_down_SOW_OTROSBCOS_MAX_historia
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3252336448598131  PRECISION:  0.04874632301442779  F1:  0.08478499208186137  GINI:  -0.061365467060942924
//////////////////////////////////////////////////
>>  _0.3_cut_down_SOW_OTROSBCOS_MAX_historia
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.22897196261682243  PRECISION:  0.04168793602177982  F1:  0.07053404347200232  GINI:  -0.09164741572770685
//////////////////////////////////////////////////


In [169]:
var = vars_top[85]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TENDENCIA_IMPORTE_LAB_U7M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TENDENCIA_IMPORTE_LAB_U7M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TENDENCIA_IMPORTE_LAB_U7M,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [170]:
var = vars_top[86]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_LAB_PROMEDIO_U9M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_LAB_PROMEDIO_U9M,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,IMPORTE_LAB_PROMEDIO_U9M,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [171]:
var = vars_top[87]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3,missing,996,0.720174,938,58,0.058233,-0.429554,0.111402,0.29218,missing,True
1,TIENE_IMPORTE_LAB_U1M_OVER_U12M_MAYORIGUAL_1.3,"[-inf,inf)",387,0.279826,325,62,0.160207,0.697063,0.180778,0.29218,inf,False


In [172]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC', cortes=[1, 2], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1766355140186916  PRECISION:  0.1801715919923737  F1:  0.17838603114676738  GINI:  0.12767729928568472
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.017757009345794394  PRECISION:  0.34545454545454546  F1:  0.03377777777777778  GINI:  0.01570759570580793
//////////////////////////////////////////////////


In [173]:
var = vars_top[88]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_IBK


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_IBK,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.288946,missing,True
1,PERCENT_GAR_IBK,"[-inf,0.8)",965,0.697758,860,105,0.108808,0.250781,0.048672,0.288946,0.8,False
2,PERCENT_GAR_IBK,"[0.8,1.0)",75,0.054230,69,6,0.080000,-0.088594,0.000410,0.288946,1.0,False
3,PERCENT_GAR_IBK,"[1.0,inf)",296,0.214027,288,8,0.027027,-1.229766,0.198437,0.288946,inf,False


In [174]:
var = vars_top[89]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_SF,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.287965,missing,True
1,MAX_DIAS_ATRASO_SF,"[-inf,1.0)",1216,0.879248,1121,95,0.078125,-0.114346,0.010966,0.287965,1.0,False
2,MAX_DIAS_ATRASO_SF,"[1.0,inf)",71,0.051338,49,22,0.309859,1.552976,0.224462,0.287965,inf,False


In [175]:
#data = add_new_var_x_cut_up(data, colum='DEBT_ratio_AVG_TICKET_DECREASE_u6m_yoy', cortes=[1], impute=0, up=True, soft=False)

In [176]:
var = vars_top[90]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_SFSF_ULT_RCC_y


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_SFSF_ULT_RCC_y,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.287965,missing,True
1,MAX_DIAS_ATRASO_SFSF_ULT_RCC_y,"[-inf,1.0)",1216,0.879248,1121,95,0.078125,-0.114346,0.010966,0.287965,1.0,False
2,MAX_DIAS_ATRASO_SFSF_ULT_RCC_y,"[1.0,inf)",71,0.051338,49,22,0.309859,1.552976,0.224462,0.287965,inf,False


In [177]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_MENOR_DE_U3M', cortes=[6], impute=0, up=True, soft=False)

>>  _6_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.03177570093457944  PRECISION:  0.08415841584158416  F1:  0.04613297150610583  GINI:  0.010712282968053266
//////////////////////////////////////////////////


In [178]:
var = vars_top[91]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.287102,missing,True
1,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[-inf,1000000.0)",412,0.297903,380,32,0.077670,-0.120682,0.004128,0.287102,1000000.0,False
2,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[1000000.0,1500000.0)",105,0.075922,91,14,0.133333,0.481951,0.021503,0.287102,1500000.0,False
3,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[1500000.0,2500000.0)",220,0.159074,198,22,0.100000,0.156529,0.004158,0.287102,2500000.0,False
4,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[2500000.0,3000000.0)",78,0.056399,76,2,0.025641,-1.283833,0.055856,0.287102,3000000.0,False
5,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[3000000.0,4500000.0)",148,0.107014,130,18,0.121622,0.376591,0.017726,0.287102,4500000.0,False
6,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[4500000.0,6000000.0)",101,0.073030,98,3,0.029703,-1.132602,0.059567,0.287102,6000000.0,False
7,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[6000000.0,9000000.0)",86,0.062184,70,16,0.186047,0.877847,0.068393,0.287102,9000000.0,False
8,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[9000000.0,inf)",137,0.099060,127,10,0.072993,-0.187849,0.003235,0.287102,inf,False


In [179]:
var = vars_top[92]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_IN_SF_ULT_RCC_x


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_IN_SF_ULT_RCC_x,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.287102,missing,True
1,DEBT_IN_SF_ULT_RCC_x,"[-inf,1000000.0)",412,0.297903,380,32,0.077670,-0.120682,0.004128,0.287102,1000000.0,False
2,DEBT_IN_SF_ULT_RCC_x,"[1000000.0,1500000.0)",105,0.075922,91,14,0.133333,0.481951,0.021503,0.287102,1500000.0,False
3,DEBT_IN_SF_ULT_RCC_x,"[1500000.0,2500000.0)",220,0.159074,198,22,0.100000,0.156529,0.004158,0.287102,2500000.0,False
4,DEBT_IN_SF_ULT_RCC_x,"[2500000.0,3000000.0)",78,0.056399,76,2,0.025641,-1.283833,0.055856,0.287102,3000000.0,False
5,DEBT_IN_SF_ULT_RCC_x,"[3000000.0,4500000.0)",148,0.107014,130,18,0.121622,0.376591,0.017726,0.287102,4500000.0,False
6,DEBT_IN_SF_ULT_RCC_x,"[4500000.0,6000000.0)",101,0.073030,98,3,0.029703,-1.132602,0.059567,0.287102,6000000.0,False
7,DEBT_IN_SF_ULT_RCC_x,"[6000000.0,9000000.0)",86,0.062184,70,16,0.186047,0.877847,0.068393,0.287102,9000000.0,False
8,DEBT_IN_SF_ULT_RCC_x,"[9000000.0,inf)",137,0.099060,127,10,0.072993,-0.187849,0.003235,0.287102,inf,False


In [180]:
var = vars_top[93]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.287102,missing,True
1,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[-inf,1000000.0)",412,0.297903,380,32,0.077670,-0.120682,0.004128,0.287102,1000000.0,False
2,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[1000000.0,1500000.0)",105,0.075922,91,14,0.133333,0.481951,0.021503,0.287102,1500000.0,False
3,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[1500000.0,2500000.0)",220,0.159074,198,22,0.100000,0.156529,0.004158,0.287102,2500000.0,False
4,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[2500000.0,3000000.0)",78,0.056399,76,2,0.025641,-1.283833,0.055856,0.287102,3000000.0,False
5,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[3000000.0,4500000.0)",148,0.107014,130,18,0.121622,0.376591,0.017726,0.287102,4500000.0,False
6,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[4500000.0,6000000.0)",101,0.073030,98,3,0.029703,-1.132602,0.059567,0.287102,6000000.0,False
7,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[6000000.0,9000000.0)",86,0.062184,70,16,0.186047,0.877847,0.068393,0.287102,9000000.0,False
8,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[9000000.0,inf)",137,0.099060,127,10,0.072993,-0.187849,0.003235,0.287102,inf,False


In [181]:
var = vars_top[94]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.287102,missing,True
1,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[-inf,1000000.0)",412,0.297903,380,32,0.077670,-0.120682,0.004128,0.287102,1000000.0,False
2,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[1000000.0,1500000.0)",105,0.075922,91,14,0.133333,0.481951,0.021503,0.287102,1500000.0,False
3,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[1500000.0,2500000.0)",220,0.159074,198,22,0.100000,0.156529,0.004158,0.287102,2500000.0,False
4,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[2500000.0,3000000.0)",78,0.056399,76,2,0.025641,-1.283833,0.055856,0.287102,3000000.0,False
5,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[3000000.0,4500000.0)",148,0.107014,130,18,0.121622,0.376591,0.017726,0.287102,4500000.0,False
6,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[4500000.0,6000000.0)",101,0.073030,98,3,0.029703,-1.132602,0.059567,0.287102,6000000.0,False
7,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[6000000.0,9000000.0)",86,0.062184,70,16,0.186047,0.877847,0.068393,0.287102,9000000.0,False
8,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[9000000.0,inf)",137,0.099060,127,10,0.072993,-0.187849,0.003235,0.287102,inf,False


In [182]:
var = vars_top[95]
print(">>>>> ", var)
display(cortes[var])

>>>>>  SOW_IBK_MAX_historia


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SOW_IBK_MAX_historia,missing,17,0.012292,17,0,0.000000,-0.592292,0.003563,0.282488,missing,True
1,SOW_IBK_MAX_historia,"[-inf,0.2)",489,0.353579,447,42,0.085890,-0.018608,0.000121,0.282488,0.2,False
2,SOW_IBK_MAX_historia,"[0.2,0.30000000000000004)",115,0.083153,94,21,0.182609,0.847509,0.084133,0.282488,0.30000000000000004,False
3,SOW_IBK_MAX_historia,"[0.30000000000000004,0.7000000000000001)",231,0.167028,209,22,0.095238,0.094990,0.001566,0.282488,0.7000000000000001,False
4,SOW_IBK_MAX_historia,"[0.7000000000000001,1.0)",197,0.142444,173,24,0.121827,0.371044,0.022832,0.282488,1.0,False
5,SOW_IBK_MAX_historia,"[1.0,inf)",334,0.241504,323,11,0.032934,-1.033476,0.170271,0.282488,inf,False


In [183]:
data = add_new_var_x_cut_up(data, colum='TENDENCIA_PROTESTOS_U7M', cortes=[0], impute=0, up=True, soft=True)

>>  _0_cut_up_TENDENCIA_PROTESTOS_U7M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.03457943925233645  PRECISION:  0.06788990825688074  F1:  0.045820433436532505  GINI:  0.005659935665862648
//////////////////////////////////////////////////


In [184]:
var = vars_top[96]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_AUTO_HIPO_PREN


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_AUTO_HIPO_PREN,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.282009,missing,True
1,PERCENT_GAR_AUTO_HIPO_PREN,"[-inf,0.1)",486,0.351410,458,28,0.057613,-0.440911,0.057008,0.282009,0.1,False
2,PERCENT_GAR_AUTO_HIPO_PREN,"[0.1,0.30000000000000004)",259,0.187274,237,22,0.084942,-0.023264,0.000100,0.282009,0.30000000000000004,False
3,PERCENT_GAR_AUTO_HIPO_PREN,"[0.30000000000000004,0.4)",107,0.077368,84,23,0.214953,1.058431,0.132471,0.282009,0.4,False
4,PERCENT_GAR_AUTO_HIPO_PREN,"[0.4,0.5)",113,0.081706,100,13,0.115044,0.313533,0.009142,0.282009,0.5,False
5,PERCENT_GAR_AUTO_HIPO_PREN,"[0.5,0.6000000000000001)",91,0.065799,87,4,0.043956,-0.725860,0.025805,0.282009,0.6000000000000001,False
6,PERCENT_GAR_AUTO_HIPO_PREN,"[0.6000000000000001,0.8)",168,0.121475,153,15,0.089286,0.031366,0.000121,0.282009,0.8,False
7,PERCENT_GAR_AUTO_HIPO_PREN,"[0.8,inf)",112,0.080983,98,14,0.125000,0.407843,0.015936,0.282009,inf,False


In [185]:
data = add_new_var_x_cut_up(data, colum='TRX_PAGOS_MASIVOS__CASH_OUT_PERCENT_U3M', cortes=[0.6], impute=0, up=True, soft=False)

>>  _0.6_cut_up_TRX_PAGOS_MASIVOS__CASH_OUT_PERCENT_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09158878504672897  PRECISION:  0.05384615384615385  F1:  0.06782006920415225  GINI:  -0.006441500732617511
//////////////////////////////////////////////////


In [186]:
var = vars_top[97]
print(">>>>> ", var)
display(cortes[var])

>>>>>  VENTAS_NETAS_ANUAL_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,VENTAS_NETAS_ANUAL_CEF_SITUACION,missing,285,0.206074,261,24,0.084211,-0.032713,0.000218,0.28128,missing,True
1,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[-inf,7000.0)",540,0.390456,476,64,0.118519,0.347219,0.054323,0.28128,7000.0,False
2,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[7000.0,9000.0)",104,0.075199,102,2,0.019231,-1.578072,0.101144,0.28128,9000.0,False
3,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[9000.0,21000.0)",304,0.219812,278,26,0.085526,-0.015771,0.000054,0.28128,21000.0,False
4,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[21000.0,33000.0)",74,0.053507,71,3,0.040541,-0.810314,0.025294,0.28128,33000.0,False
5,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[33000.0,inf)",76,0.054953,75,1,0.013158,-1.963735,0.100247,0.28128,inf,False


In [187]:
var = vars_top[98]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION,missing,331,0.239335,306,25,0.075529,-0.150956,0.005124,0.280256,missing,True
1,ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION,"[-inf,-5.0)",205,0.148228,196,9,0.043902,-0.727137,0.058306,0.280256,-5.0,False
2,ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION,"[-5.0,0.0)",79,0.057122,77,2,0.025316,-1.296905,0.057452,0.280256,0.0,False
3,ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION,"[0.0,5.0)",225,0.162690,210,15,0.066667,-0.285304,0.011775,0.280256,5.0,False
4,ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION,"[5.0,65.0)",359,0.259581,324,35,0.097493,0.128358,0.004510,0.280256,65.0,False
5,ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION,"[65.0,inf)",184,0.133044,150,34,0.184783,0.869479,0.143089,0.280256,inf,False


In [188]:
var = vars_top[99]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO,missing,962,0.695589,906,56,0.058212,-0.429934,0.107773,0.277473,missing,True
1,ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO,"[-inf,0.1)",252,0.182213,218,34,0.134921,0.495619,0.054879,0.277473,0.1,False
2,ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO,"[0.1,inf)",169,0.122198,139,30,0.177515,0.820477,0.114821,0.277473,inf,False


In [189]:
var = vars_top[100]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT_SCOTIA


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT_SCOTIA,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.276976,missing,True
1,GAR_TOT_SCOTIA,"[-inf,700000.0)",982,0.710051,899,83,0.084521,-0.028689,0.000578,0.276976,700000.0,False
2,GAR_TOT_SCOTIA,"[700000.0,1650000.0)",127,0.091829,125,2,0.015748,-1.781413,0.146617,0.276976,1650000.0,False
3,GAR_TOT_SCOTIA,"[1650000.0,3050000.0)",76,0.054953,62,14,0.184211,0.865676,0.058500,0.276976,3050000.0,False
4,GAR_TOT_SCOTIA,"[3050000.0,inf)",151,0.109183,131,20,0.132450,0.474288,0.029854,0.276976,inf,False


In [190]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M', cortes=[1, 2, 3], impute=0, up=False, soft=True)

>>  _1_cut_down_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5906542056074766  PRECISION:  0.048694044225287  F1:  0.08997081642821553  GINI:  -0.11223774475117076
//////////////////////////////////////////////////
>>  _2_cut_down_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6831775700934579  PRECISION:  0.05023364485981308  F1:  0.09358596850595313  GINI:  -0.10362648319129675
//////////////////////////////////////////////////
>>  _3_cut_down_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.888785046728972  PRECISION:  0.05546159678077798  F1:  0.10440797057693364  GINI:  -0.03322337863821456
//////////////////////////////////////////////////


In [191]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT', cortes=[0, -6], impute=0, up=False, soft=True)

>>  _0_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.44672897196261685  PRECISION:  0.06992393212404915  F1:  0.12092081963066027  GINI:  0.08477975187836306
//////////////////////////////////////////////////
>>  _-6_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.18317757009345795  PRECISION:  0.0771957463568334  F1:  0.1086173455250762  GINI:  0.049794899024347083
//////////////////////////////////////////////////


In [192]:
var = vars_top[101]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ROTACION_INVENTARIOS_VAR_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ROTACION_INVENTARIOS_VAR_CEF_SITUACION,missing,285,0.206074,261,24,0.084211,-0.032713,0.000218,0.276928,missing,True
1,ROTACION_INVENTARIOS_VAR_CEF_SITUACION,"[-inf,0.0)",283,0.204628,272,11,0.038869,-0.854153,0.105653,0.276928,0.0,False
2,ROTACION_INVENTARIOS_VAR_CEF_SITUACION,"[0.0,1.0)",492,0.355748,458,34,0.069106,-0.246755,0.019567,0.276928,1.0,False
3,ROTACION_INVENTARIOS_VAR_CEF_SITUACION,"[1.0,3.0)",218,0.157628,187,31,0.142202,0.556632,0.061382,0.276928,3.0,False
4,ROTACION_INVENTARIOS_VAR_CEF_SITUACION,"[3.0,inf)",105,0.075922,85,20,0.190476,0.906834,0.090109,0.276928,inf,False


In [193]:
var = vars_top[102]
print(">>>>> ", var)
display(cortes[var])

>>>>>  RECENCY_historia


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,RECENCY_historia,missing,232,0.167751,206,26,0.112069,0.283974,0.015210,0.275078,missing,True
1,RECENCY_historia,"[-inf,3.0)",279,0.201735,246,33,0.118280,0.344929,0.027672,0.275078,3.0,False
2,RECENCY_historia,"[3.0,15.0)",573,0.414317,548,25,0.043630,-0.733646,0.165477,0.275078,15.0,False
3,RECENCY_historia,"[15.0,22.0)",95,0.068691,87,8,0.084211,-0.032713,0.000073,0.275078,22.0,False
4,RECENCY_historia,"[22.0,31.0)",75,0.054230,61,14,0.186667,0.881937,0.060297,0.275078,31.0,False
5,RECENCY_historia,"[31.0,inf)",129,0.093275,115,14,0.108527,0.247879,0.006349,0.275078,inf,False


In [194]:
texto_a_comparar = """
AYACUCHO%,%SAN MARTIN%,%MOQUEGUA%,%PIURA%,%TUMBES%,%HUANCAVELICA
""".upper()

otro_texto_adicional = """
LIMA%,%JUNIN%,%AMAZONAS%,%UCAYALI%,%ICA%,%LORETO%,%CUSCO%,%ANCASH
""".upper()

data['DEPARTAMENTOletal__s1_provincia'] = data['DEPARTAMENTO'].apply(lambda _: None if pd.isnull(_) else (1 if _.upper() in texto_a_comparar.upper() else 0))
data['departamento_decuidado'] = data['DEPARTAMENTO'].apply(
    lambda _: None if pd.isnull(_) else (1 if _.upper() in texto_a_comparar or _.upper() in otro_texto_adicional.upper() else 0))

In [195]:
var = vars_top[103]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CAJA_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CAJA_CEF_SITUACION,missing,285,0.206074,261,24,0.084211,-0.032713,0.000218,0.274332,missing,True
1,CAJA_CEF_SITUACION,"[-inf,800.0)",494,0.357195,433,61,0.123482,0.393890,0.065188,0.274332,800.0,False
2,CAJA_CEF_SITUACION,"[800.0,1300.0)",186,0.134490,181,5,0.026882,-1.235306,0.125560,0.274332,1300.0,False
3,CAJA_CEF_SITUACION,"[1300.0,1800.0)",131,0.094722,120,11,0.083969,-0.035843,0.000120,0.274332,1800.0,False
4,CAJA_CEF_SITUACION,"[1800.0,2700.0)",127,0.091829,121,6,0.047244,-0.650278,0.029785,0.274332,2700.0,False
5,CAJA_CEF_SITUACION,"[2700.0,5100.0)",75,0.054230,65,10,0.133333,0.481951,0.015359,0.274332,5100.0,False
6,CAJA_CEF_SITUACION,"[5100.0,inf)",85,0.061461,82,3,0.035294,-0.954354,0.038102,0.274332,inf,False


In [196]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_DEBT_MAYOR_DE_U3M', cortes=[1, 2, 3], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4093457943925234  PRECISION:  0.07742619763125332  F1:  0.1302214954660324  GINI:  0.11223774475117088
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11121495327102804  PRECISION:  0.07991940899932841  F1:  0.09300508010941774  GINI:  0.03322337863821456
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_BANCOS_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.020560747663551402  PRECISION:  0.06162464985994398  F1:  0.03083391730903995  GINI:  0.0014898151803450776
//////////////////////////////////////////////////


In [197]:
var = vars_top[104]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO,missing,279,0.201735,266,13,0.046595,-0.664794,0.067993,0.271751,missing,True
1,MAX_DIAS_ATRASO,"[-inf,1.0)",995,0.719450,915,80,0.080402,-0.083144,0.004806,0.271751,1.0,False
2,MAX_DIAS_ATRASO,"[1.0,inf)",109,0.078814,82,27,0.247706,1.242871,0.198953,0.271751,inf,False


In [198]:
var = vars_top[105]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x,missing,279,0.201735,266,13,0.046595,-0.664794,0.067993,0.271751,missing,True
1,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x,"[-inf,1.0)",995,0.719450,915,80,0.080402,-0.083144,0.004806,0.271751,1.0,False
2,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x,"[1.0,inf)",109,0.078814,82,27,0.247706,1.242871,0.198953,0.271751,inf,False


In [199]:
var = vars_top[106]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_y


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_y,missing,279,0.201735,266,13,0.046595,-0.664794,0.067993,0.271751,missing,True
1,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_y,"[-inf,1.0)",995,0.719450,915,80,0.080402,-0.083144,0.004806,0.271751,1.0,False
2,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_y,"[1.0,inf)",109,0.078814,82,27,0.247706,1.242871,0.198953,0.271751,inf,False


In [200]:
var = vars_top[107]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,missing,285,0.206074,261,24,0.084211,-0.032713,0.000218,0.270668,missing,True
1,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[-inf,0.10000000000000003)",80,0.057845,74,6,0.075000,-0.158552,0.001362,0.270668,0.10000000000000003,False
2,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.10000000000000003,0.2)",106,0.076645,104,2,0.018868,-1.597490,0.104918,0.270668,0.2,False
3,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.2,0.3)",111,0.080260,106,5,0.045045,-0.700248,0.029593,0.270668,0.3,False
4,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.3,0.4000000000000001)",124,0.089660,112,12,0.096774,0.120161,0.001360,0.270668,0.4000000000000001,False
5,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.4000000000000001,0.5)",126,0.091106,122,4,0.031746,-1.063973,0.067309,0.270668,0.5,False
6,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.5,inf)",551,0.398409,484,67,0.121597,0.376361,0.065908,0.270668,inf,False


In [201]:
var = vars_top[108]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_HIPO_TOT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_HIPO_TOT,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.266654,missing,True
1,GAR_HIPO_TOT,"[-inf,600000.0)",712,0.514823,656,56,0.078652,-0.107056,0.005645,0.266654,600000.0,False
2,GAR_HIPO_TOT,"[600000.0,1800000.0)",227,0.164136,215,12,0.052863,-0.531978,0.037361,0.266654,1800000.0,False
3,GAR_HIPO_TOT,"[1800000.0,3000000.0)",103,0.074476,80,23,0.223301,1.107221,0.142085,0.266654,3000000.0,False
4,GAR_HIPO_TOT,"[3000000.0,5000000.0)",124,0.089660,107,17,0.137097,0.514138,0.029279,0.266654,5000000.0,False
5,GAR_HIPO_TOT,"[5000000.0,inf)",170,0.122921,159,11,0.064706,-0.317256,0.010858,0.266654,inf,False


In [202]:
var = vars_top[109]
print(">>>>> ", var)
display(cortes[var])

>>>>>  SOW_IBK_MIN_historia


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SOW_IBK_MIN_historia,missing,17,0.012292,17,0,0.000000,-0.592292,0.003563,0.266212,missing,True
1,SOW_IBK_MIN_historia,"[-inf,0.1)",591,0.427332,539,52,0.087986,0.007810,0.000026,0.266212,0.1,False
2,SOW_IBK_MIN_historia,"[0.1,1.0)",558,0.403471,494,64,0.114695,0.302629,0.041833,0.266212,1.0,False
3,SOW_IBK_MIN_historia,"[1.0,inf)",217,0.156905,213,4,0.018433,-1.628716,0.220790,0.266212,inf,False


In [203]:
var = vars_top[110]
print(">>>>> ", var)
display(cortes[var])

>>>>>  minimo_percent_ibk_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,minimo_percent_ibk_u12m,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.265442,missing,True
1,minimo_percent_ibk_u12m,"[-inf,0.5)",980,0.708604,882,98,0.100000,0.156529,0.018522,0.265442,0.5,False
2,minimo_percent_ibk_u12m,"[0.5,1.0)",96,0.069414,82,14,0.145833,0.586091,0.030325,0.265442,1.0,False
3,minimo_percent_ibk_u12m,"[1.0,inf)",260,0.187997,253,7,0.026923,-1.233726,0.175169,0.265442,inf,False


In [204]:
var = vars_top[111]
print(">>>>> ", var)
display(cortes[var])

>>>>>  minimo_percent_sow_ibk_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,minimo_percent_sow_ibk_u12m,missing,25,0.018077,24,1,0.040000,-0.824300,0.008794,0.262016,missing,True
1,minimo_percent_sow_ibk_u12m,"[-inf,0.7000000000000001)",1082,0.782357,971,111,0.102588,0.184957,0.028889,0.262016,0.7000000000000001,False
2,minimo_percent_sow_ibk_u12m,"[0.7000000000000001,1.0)",89,0.064353,84,5,0.056180,-0.467626,0.011617,0.262016,1.0,False
3,minimo_percent_sow_ibk_u12m,"[1.0,inf)",187,0.135213,184,3,0.016043,-1.762570,0.212716,0.262016,inf,False


In [205]:
var = vars_top[112]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PROM_CANT_TRX_U6M_CASHIN


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PROM_CANT_TRX_U6M_CASHIN,missing,102,0.073753,92,10,0.098039,0.134550,0.001412,0.260508,missing,True
1,PROM_CANT_TRX_U6M_CASHIN,"[-inf,1.0)",306,0.221258,269,37,0.120915,0.369960,0.035275,0.260508,1.0,False
2,PROM_CANT_TRX_U6M_CASHIN,"[1.0,2.0)",207,0.149675,188,19,0.091787,0.061750,0.000585,0.260508,2.0,False
3,PROM_CANT_TRX_U6M_CASHIN,"[2.0,3.0)",116,0.083876,113,3,0.025862,-1.275022,0.082200,0.260508,3.0,False
4,PROM_CANT_TRX_U6M_CASHIN,"[3.0,4.0)",79,0.057122,70,9,0.113924,0.302483,0.005922,0.260508,4.0,False
5,PROM_CANT_TRX_U6M_CASHIN,"[4.0,7.0)",167,0.120752,158,9,0.053892,-0.511617,0.025631,0.260508,7.0,False
6,PROM_CANT_TRX_U6M_CASHIN,"[7.0,9.0)",78,0.056399,65,13,0.166667,0.744315,0.042328,0.260508,9.0,False
7,PROM_CANT_TRX_U6M_CASHIN,"[9.0,33.0)",224,0.161967,207,17,0.075893,-0.145752,0.003240,0.260508,33.0,False
8,PROM_CANT_TRX_U6M_CASHIN,"[33.0,inf)",104,0.075199,101,3,0.028846,-1.162755,0.063915,0.260508,inf,False


In [206]:
var = vars_top[113]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CANTIDAD_CASHOUT_U1M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CANTIDAD_CASHOUT_U1M,missing,95,0.068691,85,10,0.105263,0.213687,0.003426,0.25922,missing,True
1,CANTIDAD_CASHOUT_U1M,"[-inf,5.0)",400,0.289226,358,42,0.105000,0.210890,0.014034,0.25922,5.0,False
2,CANTIDAD_CASHOUT_U1M,"[5.0,20.0)",352,0.254519,331,21,0.059659,-0.403843,0.035164,0.25922,20.0,False
3,CANTIDAD_CASHOUT_U1M,"[20.0,25.0)",79,0.057122,70,9,0.113924,0.302483,0.005922,0.25922,25.0,False
4,CANTIDAD_CASHOUT_U1M,"[25.0,45.0)",122,0.088214,101,21,0.172131,0.783155,0.074425,0.25922,45.0,False
5,CANTIDAD_CASHOUT_U1M,"[45.0,105.0)",149,0.107737,137,12,0.080537,-0.081321,0.000689,0.25922,105.0,False
6,CANTIDAD_CASHOUT_U1M,"[105.0,inf)",186,0.134490,181,5,0.026882,-1.235306,0.125560,0.25922,inf,False


In [207]:
var = vars_top[114]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CANT_TRX_CASHOUT_MES_ACTUAL


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CANT_TRX_CASHOUT_MES_ACTUAL,missing,95,0.068691,85,10,0.105263,0.213687,0.003426,0.25922,missing,True
1,CANT_TRX_CASHOUT_MES_ACTUAL,"[-inf,5.0)",400,0.289226,358,42,0.105000,0.210890,0.014034,0.25922,5.0,False
2,CANT_TRX_CASHOUT_MES_ACTUAL,"[5.0,20.0)",352,0.254519,331,21,0.059659,-0.403843,0.035164,0.25922,20.0,False
3,CANT_TRX_CASHOUT_MES_ACTUAL,"[20.0,25.0)",79,0.057122,70,9,0.113924,0.302483,0.005922,0.25922,25.0,False
4,CANT_TRX_CASHOUT_MES_ACTUAL,"[25.0,45.0)",122,0.088214,101,21,0.172131,0.783155,0.074425,0.25922,45.0,False
5,CANT_TRX_CASHOUT_MES_ACTUAL,"[45.0,105.0)",149,0.107737,137,12,0.080537,-0.081321,0.000689,0.25922,105.0,False
6,CANT_TRX_CASHOUT_MES_ACTUAL,"[105.0,inf)",186,0.134490,181,5,0.026882,-1.235306,0.125560,0.25922,inf,False


In [208]:
var = vars_top[115]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_DEUDA_FULL_RCC_BCP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_DEUDA_FULL_RCC_BCP,missing,6,0.004338,6,0,0.000000,0.449161,0.001210,0.258338,missing,True
1,MTO_DEUDA_FULL_RCC_BCP,"[-inf,200000.0)",706,0.510484,651,55,0.077904,-0.124895,0.007558,0.258338,200000.0,False
2,MTO_DEUDA_FULL_RCC_BCP,"[200000.0,2000000.0)",253,0.182936,239,14,0.055336,-0.491125,0.036065,0.258338,2000000.0,False
3,MTO_DEUDA_FULL_RCC_BCP,"[2000000.0,2800000.0)",70,0.050615,52,18,0.257143,1.285409,0.138453,0.258338,2800000.0,False
4,MTO_DEUDA_FULL_RCC_BCP,"[2800000.0,4400000.0)",88,0.063630,81,7,0.079545,-0.102258,0.000637,0.258338,4400000.0,False
5,MTO_DEUDA_FULL_RCC_BCP,"[4400000.0,7200000.0)",92,0.066522,76,16,0.173913,0.788137,0.056877,0.258338,7200000.0,False
6,MTO_DEUDA_FULL_RCC_BCP,"[7200000.0,inf)",168,0.121475,158,10,0.059524,-0.413729,0.017536,0.258338,inf,False


In [209]:
var = vars_top[116]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TMO_DEUDA_TOTAL_FULL_RCC_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TMO_DEUDA_TOTAL_FULL_RCC_SF,missing,6,0.004338,6,0,0.000000,0.449161,0.001210,0.254904,missing,True
1,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[-inf,2000000.0)",294,0.212581,273,21,0.071429,-0.218668,0.009283,0.254904,2000000.0,False
2,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[2000000.0,4000000.0)",145,0.104845,140,5,0.034483,-0.985923,0.068512,0.254904,4000000.0,False
3,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[4000000.0,6000000.0)",87,0.062907,74,13,0.149425,0.607166,0.029712,0.254904,6000000.0,False
4,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[6000000.0,14000000.0)",389,0.281273,351,38,0.097686,0.123081,0.004480,0.254904,14000000.0,False
5,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[14000000.0,20000000.0)",84,0.060738,71,13,0.154762,0.648551,0.033278,0.254904,20000000.0,False
6,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[20000000.0,26000000.0)",120,0.086768,116,4,0.033333,-1.021014,0.059994,0.254904,26000000.0,False
7,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[26000000.0,62000000.0)",172,0.124367,150,22,0.127907,0.426689,0.026968,0.254904,62000000.0,False
8,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[62000000.0,inf)",86,0.062184,82,4,0.046512,-0.674144,0.021464,0.254904,inf,False


In [210]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF', cortes=[5, 6], impute=0, up=True, soft=False)

>>  _5_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.048598130841121495  PRECISION:  0.29714285714285715  F1:  0.08353413654618473  GINI:  0.041595967571168124
//////////////////////////////////////////////////
>>  _6_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.016822429906542057  PRECISION:  0.2857142857142857  F1:  0.03177405119152692  GINI:  0.014260662856559136
//////////////////////////////////////////////////


In [211]:
var = vars_top[117]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_HIPO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_HIPO,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.252606,missing,True
1,PERCENT_GAR_HIPO,"[-inf,0.1)",687,0.496746,636,51,0.074236,-0.169620,0.013326,0.252606,0.1,False
2,PERCENT_GAR_HIPO,"[0.1,0.2)",105,0.075922,95,10,0.095238,0.102462,0.000832,0.252606,0.2,False
3,PERCENT_GAR_HIPO,"[0.2,0.30000000000000004)",121,0.087491,111,10,0.082645,-0.053192,0.000242,0.252606,0.30000000000000004,False
4,PERCENT_GAR_HIPO,"[0.30000000000000004,0.4)",101,0.073030,82,19,0.188119,0.891473,0.083271,0.252606,0.4,False
5,PERCENT_GAR_HIPO,"[0.4,0.6000000000000001)",127,0.091829,123,4,0.031496,-1.072137,0.068674,0.252606,0.6000000000000001,False
6,PERCENT_GAR_HIPO,"[0.6000000000000001,0.8)",124,0.089660,111,13,0.104839,0.209173,0.004277,0.252606,0.8,False
7,PERCENT_GAR_HIPO,"[0.8,inf)",71,0.051338,59,12,0.169014,0.761123,0.040557,0.252606,inf,False


In [212]:
data = add_new_var_x_cut_up(data, colum='NRO_VECES_REACTIVA_IBK', cortes=[2], impute=0, up=True, soft=False)

>>  _2_cut_up_NUM_VECES_REACTIVA_IBK
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.15140186915887852  PRECISION:  0.05988909426987061  F1:  0.08582781456953642  GINI:  0.0066335667565102785
//////////////////////////////////////////////////


In [213]:
var = vars_top[118]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_IN__BCPSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_IN__BCPSF_ULT_RCC,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.252552,missing,True
1,DEBT_IN__BCPSF_ULT_RCC,"[-inf,150000.0)",739,0.534346,678,61,0.082544,-0.054520,0.001553,0.252552,150000.0,False
2,DEBT_IN__BCPSF_ULT_RCC,"[150000.0,350000.0)",92,0.066522,89,3,0.032609,-1.036271,0.047116,0.252552,350000.0,False
3,DEBT_IN__BCPSF_ULT_RCC,"[350000.0,600000.0)",89,0.064353,72,17,0.191011,0.910301,0.077066,0.252552,600000.0,False
4,DEBT_IN__BCPSF_ULT_RCC,"[600000.0,1750000.0)",197,0.142444,170,27,0.137056,0.513792,0.046447,0.252552,1750000.0,False
5,DEBT_IN__BCPSF_ULT_RCC,"[1750000.0,inf)",170,0.122921,161,9,0.052941,-0.530426,0.027834,0.252552,inf,False


In [214]:
var = vars_top[119]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MONTO_LEASING_VIGENTE_U2M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MONTO_LEASING_VIGENTE_U2M,missing,103,0.074476,96,7,0.067961,-0.264685,0.004679,0.251764,missing,True
1,MONTO_LEASING_VIGENTE_U2M,"[-inf,20000.0)",552,0.399132,501,51,0.092391,0.068973,0.001954,0.251764,20000.0,False
2,MONTO_LEASING_VIGENTE_U2M,"[20000.0,40000.0)",79,0.057122,70,9,0.113924,0.302483,0.005922,0.251764,40000.0,False
3,MONTO_LEASING_VIGENTE_U2M,"[40000.0,160000.0)",197,0.142444,190,7,0.035533,-0.947361,0.087254,0.251764,160000.0,False
4,MONTO_LEASING_VIGENTE_U2M,"[160000.0,380000.0)",187,0.135213,173,14,0.074866,-0.160481,0.003259,0.251764,380000.0,False
5,MONTO_LEASING_VIGENTE_U2M,"[380000.0,680000.0)",94,0.067968,91,3,0.031915,-1.058494,0.049803,0.251764,680000.0,False
6,MONTO_LEASING_VIGENTE_U2M,"[680000.0,inf)",171,0.123644,142,29,0.169591,0.765222,0.098894,0.251764,inf,False


In [215]:
var = vars_top[120]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NUMERO_TRABAJADORES_REGISTRADOS_U10M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NUMERO_TRABAJADORES_REGISTRADOS_U10M,missing,32,0.023138,30,2,0.062500,-0.354297,0.002511,0.249296,missing,True
1,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[-inf,5.0)",287,0.207520,268,19,0.066202,-0.292795,0.015770,0.249296,5.0,False
2,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[5.0,15.0)",305,0.220535,271,34,0.111475,0.277995,0.019116,0.249296,15.0,False
3,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[15.0,40.0)",366,0.264642,332,34,0.092896,0.074979,0.001535,0.249296,40.0,False
4,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[40.0,75.0)",126,0.091106,104,22,0.174603,0.800405,0.080833,0.249296,75.0,False
5,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[75.0,inf)",267,0.193059,258,9,0.033708,-1.001982,0.129532,0.249296,inf,False


In [216]:
texto_a_comparar = """
menos del 10%
""".upper()

otro_texto_adicional = """
entre10 y 20%%
""".upper()

data['_20percent_down_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC'] = data['cat_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC'].apply(
    lambda _: None if pd.isnull(_) else (1 if _.upper() in texto_a_comparar or _.upper() in otro_texto_adicional.upper() else 0))

In [217]:
[col for col in data.columns if 'COVER' in col.upper()]

['COVER_GAR_AUTO_ULT_RCC',
 'COVER_GAR_AUTO_HIPO_PREN_ULT_RCC',
 'COVER_GAR_AUTO_HIPO_ULT_RCC',
 'cat_COVER_GAR_AUTO_ULT_RCC',
 'cat_COVER_GAR_AUTO_HIPO_ULT_RCC',
 'cat_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC',
 'cat_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC',
 'COVER_IBK_GAR_PREFERIDA',
 'COVER_IBK_GAR_AUTO_HIPO',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.9',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.8',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.7',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.6',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.5',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.4',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.3',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.2',
 'TIENE_COVER_IBK_GAR_PREFERIDA_MENOR_0.1',
 'TIENE_COVER_IBK_GAR_PREFERIDA_IGUAL_0',
 'TIENE_COVER_IBK_GAR_AUTO_HIPO_MENOR_0.9',
 'TIENE_COVER_IBK_GAR_AUTO_HIPO_MENOR_0.8',
 'TIENE_COVER_IBK_GAR_AUTO_HIPO_MENOR_0.7',
 'TIENE_COVER_IBK_GAR_AUTO_HIPO_MENOR_0.6',
 'TIENE_COVER_IBK_GAR_AUTO_HIPO_MENOR_0.5',
 'TIENE_COVER_IBK_GAR_AUTO_HIPO_

In [218]:
data = add_new_var_x_cut_up(data, colum='DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC', cortes=[0.7], impute=0, up=True, soft=False)

>>  _0.7_cut_up_DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.40186915887850466  PRECISION:  0.06469083797201745  F1:  0.11144227031229752  GINI:  0.047946808884197445
//////////////////////////////////////////////////


In [219]:
var = vars_top[121]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_IN__BBVASF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_IN__BBVASF_ULT_RCC,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.243934,missing,True
1,DEBT_IN__BBVASF_ULT_RCC,"[-inf,100000.0)",739,0.534346,668,71,0.096076,0.112145,0.007039,0.243934,100000.0,False
2,DEBT_IN__BBVASF_ULT_RCC,"[100000.0,450000.0)",123,0.088937,120,3,0.024390,-1.335126,0.093475,0.243934,450000.0,False
3,DEBT_IN__BBVASF_ULT_RCC,"[450000.0,700000.0)",74,0.053507,70,4,0.054054,-0.508448,0.011232,0.243934,700000.0,False
4,DEBT_IN__BBVASF_ULT_RCC,"[700000.0,1800000.0)",121,0.087491,100,21,0.173554,0.793106,0.075998,0.243934,1800000.0,False
5,DEBT_IN__BBVASF_ULT_RCC,"[1800000.0,3450000.0)",87,0.062907,81,6,0.068966,-0.248936,0.003518,0.243934,3450000.0,False
6,DEBT_IN__BBVASF_ULT_RCC,"[3450000.0,inf)",143,0.103398,131,12,0.083916,-0.036537,0.000136,0.243934,inf,False


In [220]:
var = vars_top[122]
print(">>>>> ", var)
display(cortes[var])

>>>>>  EBITDA_ANUAL_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,EBITDA_ANUAL_CEF_SITUACION,missing,285,0.206074,261,24,0.084211,-0.032713,0.000218,0.238536,missing,True
1,EBITDA_ANUAL_CEF_SITUACION,"[-inf,300.0)",222,0.160521,202,20,0.090090,0.041218,0.000277,0.238536,300.0,False
2,EBITDA_ANUAL_CEF_SITUACION,"[300.0,400.0)",94,0.067968,91,3,0.031915,-1.058494,0.049803,0.238536,400.0,False
3,EBITDA_ANUAL_CEF_SITUACION,"[400.0,800.0)",226,0.163413,195,31,0.137168,0.514741,0.053502,0.238536,800.0,False
4,EBITDA_ANUAL_CEF_SITUACION,"[800.0,1200.0)",165,0.119306,154,11,0.066667,-0.285304,0.008635,0.238536,1200.0,False
5,EBITDA_ANUAL_CEF_SITUACION,"[1200.0,1500.0)",79,0.057122,77,2,0.025316,-1.296905,0.057452,0.238536,1500.0,False
6,EBITDA_ANUAL_CEF_SITUACION,"[1500.0,2000.0)",102,0.073753,86,16,0.156863,0.671995,0.043842,0.238536,2000.0,False
7,EBITDA_ANUAL_CEF_SITUACION,"[2000.0,3700.0)",123,0.088937,114,9,0.073171,-0.185220,0.002827,0.238536,3700.0,False
8,EBITDA_ANUAL_CEF_SITUACION,"[3700.0,inf)",87,0.062907,83,4,0.045977,-0.678793,0.021981,0.238536,inf,False


In [221]:
var = vars_top[123]
print(">>>>> ", var)
display(cortes[var])

>>>>>  minimo_percent_sow_others_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,minimo_percent_sow_others_u12m,missing,25,0.018077,24,1,0.040000,-0.824300,0.008794,0.236317,missing,True
1,minimo_percent_sow_others_u12m,"[-inf,0.1)",542,0.391902,513,29,0.053506,-0.519227,0.085418,0.236317,0.1,False
2,minimo_percent_sow_others_u12m,"[0.1,0.30000000000000004)",136,0.098337,113,23,0.169118,0.761860,0.077860,0.236317,0.30000000000000004,False
3,minimo_percent_sow_others_u12m,"[0.30000000000000004,0.4)",75,0.054230,72,3,0.040000,-0.824300,0.026383,0.236317,0.4,False
4,minimo_percent_sow_others_u12m,"[0.4,0.8)",341,0.246565,300,41,0.120235,0.363543,0.037858,0.236317,0.8,False
5,minimo_percent_sow_others_u12m,"[0.8,inf)",264,0.190889,241,23,0.087121,0.004451,0.000004,0.236317,inf,False


In [222]:
var = vars_top[124]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NUMERO_TRABAJADORES_REGISTRADOS_U3M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NUMERO_TRABAJADORES_REGISTRADOS_U3M,missing,32,0.023138,30,2,0.062500,-0.354297,0.002511,0.236055,missing,True
1,NUMERO_TRABAJADORES_REGISTRADOS_U3M,"[-inf,15.0)",583,0.421547,537,46,0.078902,-0.103603,0.004335,0.236055,15.0,False
2,NUMERO_TRABAJADORES_REGISTRADOS_U3M,"[15.0,65.0)",474,0.342733,414,60,0.126582,0.422232,0.072712,0.236055,65.0,False
3,NUMERO_TRABAJADORES_REGISTRADOS_U3M,"[65.0,120.0)",89,0.064353,82,7,0.078652,-0.107056,0.000706,0.236055,120.0,False
4,NUMERO_TRABAJADORES_REGISTRADOS_U3M,"[120.0,inf)",205,0.148228,200,5,0.024390,-1.335126,0.155791,0.236055,inf,False


In [223]:
var = vars_top[125]
print(">>>>> ", var)
display(cortes[var])

>>>>>  minimo_percent_auto_hipo_pren_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,minimo_percent_auto_hipo_pren_u12m,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.234035,missing,True
1,minimo_percent_auto_hipo_pren_u12m,"[-inf,0.1)",580,0.419378,550,30,0.051724,-0.554968,0.102930,0.234035,0.1,False
2,minimo_percent_auto_hipo_pren_u12m,"[0.1,0.4)",370,0.267534,320,50,0.135135,0.497455,0.081235,0.234035,0.4,False
3,minimo_percent_auto_hipo_pren_u12m,"[0.4,inf)",386,0.279103,347,39,0.101036,0.167990,0.008443,0.234035,inf,False


In [224]:
var = vars_top[126]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M,"[-inf,1.0)",603,0.436009,547,56,0.092869,0.074656,0.002506,0.233158,1.0,False
1,NRO_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M,"[1.0,2.0)",211,0.152567,207,4,0.018957,-1.592671,0.207943,0.233158,2.0,False
2,NRO_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M,"[2.0,4.0)",422,0.305134,379,43,0.101896,0.177417,0.010335,0.233158,4.0,False
3,NRO_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M,"[4.0,inf)",147,0.106291,130,17,0.115646,0.319432,0.012374,0.233158,inf,False


In [225]:
var = vars_top[127]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_NO_PREFER


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_NO_PREFER,missing,47,0.033984,46,1,0.021277,-1.474888,4.142649e-02,0.22856,missing,True
1,PERCENT_GAR_NO_PREFER,"[-inf,0.6000000000000001)",611,0.441793,558,53,0.086743,-0.000314,4.346169e-08,0.22856,0.6000000000000001,False
2,PERCENT_GAR_NO_PREFER,"[0.6000000000000001,0.7000000000000001)",124,0.089660,98,26,0.209677,1.026882,1.428123e-01,0.22856,0.7000000000000001,False
3,PERCENT_GAR_NO_PREFER,"[0.7000000000000001,1.0)",423,0.305857,398,25,0.059102,-0.413823,4.419188e-02,0.22856,1.0,False
4,PERCENT_GAR_NO_PREFER,"[1.0,inf)",178,0.128706,163,15,0.084270,-0.031947,1.296330e-04,0.22856,inf,False


In [226]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_DEBT_MAYOR_DE_U3M', cortes=[1, 2], impute=0, up=False, soft=True)

>>  _1_cut_down_NUM_BANCOS_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5906542056074766  PRECISION:  0.048694044225287  F1:  0.08997081642821553  GINI:  -0.11223774475117076
//////////////////////////////////////////////////
>>  _2_cut_down_NUM_BANCOS_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.888785046728972  PRECISION:  0.05546159678077798  F1:  0.10440797057693364  GINI:  -0.03322337863821456
//////////////////////////////////////////////////


In [227]:
var = vars_top[128]
print(">>>>> ", var)
display(cortes[var])

>>>>>  _aux_MONTO_CASHOUT_PAGOS_MASIVOS__U11M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,_aux_MONTO_CASHOUT_PAGOS_MASIVOS__U11M,missing,95,0.068691,85,10,0.105263,0.213687,0.003426,0.226483,missing,True
1,_aux_MONTO_CASHOUT_PAGOS_MASIVOS__U11M,"[-inf,3000.0)",916,0.662328,829,87,0.094978,0.099441,0.006824,0.226483,3000.0,False
2,_aux_MONTO_CASHOUT_PAGOS_MASIVOS__U11M,"[3000.0,25000.0)",80,0.057845,75,5,0.062500,-0.354297,0.006277,0.226483,25000.0,False
3,_aux_MONTO_CASHOUT_PAGOS_MASIVOS__U11M,"[25000.0,68000.0)",79,0.057122,66,13,0.164557,0.729048,0.040883,0.226483,68000.0,False
4,_aux_MONTO_CASHOUT_PAGOS_MASIVOS__U11M,"[68000.0,inf)",213,0.154013,208,5,0.023474,-1.374347,0.169073,0.226483,inf,False


In [228]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN', cortes=[-1, -6], impute=0, up=False, soft=True)

>>  _-1_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.22429906542056074  PRECISION:  0.07626310772163966  F1:  0.11382499407161491  GINI:  0.0588089139916641
//////////////////////////////////////////////////
>>  _-6_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08971962616822429  PRECISION:  0.09293320425943853  F1:  0.09129814550641939  GINI:  0.03637794337191336
//////////////////////////////////////////////////


In [229]:
var = vars_top[129]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTOSALDO_BCP_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTOSALDO_BCP_V,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.226468,missing,True
1,MTOSALDO_BCP_V,"[-inf,500000.0)",892,0.644975,822,70,0.078475,-0.109492,0.007390,0.226468,500000.0,False
2,MTOSALDO_BCP_V,"[500000.0,1000000.0)",97,0.070137,77,20,0.206186,1.005680,0.106301,0.226468,1000000.0,False
3,MTOSALDO_BCP_V,"[1000000.0,1500000.0)",77,0.055676,70,7,0.090909,0.051168,0.000149,0.226468,1500000.0,False
4,MTOSALDO_BCP_V,"[1500000.0,2100000.0)",73,0.052784,61,12,0.164384,0.727786,0.037628,0.226468,2100000.0,False
5,MTOSALDO_BCP_V,"[2100000.0,inf)",148,0.107014,140,8,0.054054,-0.508448,0.022463,0.226468,inf,False


In [230]:
var = vars_top[130]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_TRABAJADORESX_U2M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_TRABAJADORESX_U2M,missing,32,0.023138,30,2,0.062500,-0.354297,0.002511,0.224288,missing,True
1,NRO_TRABAJADORESX_U2M,"[-inf,15.0)",578,0.417932,530,48,0.083045,-0.047923,0.000941,0.224288,15.0,False
2,NRO_TRABAJADORESX_U2M,"[15.0,60.0)",451,0.326103,396,55,0.121951,0.379672,0.054974,0.224288,60.0,False
3,NRO_TRABAJADORESX_U2M,"[60.0,120.0)",111,0.080260,101,10,0.090090,0.041218,0.000139,0.224288,120.0,False
4,NRO_TRABAJADORESX_U2M,"[120.0,inf)",211,0.152567,206,5,0.023697,-1.364685,0.165723,0.224288,inf,False


In [231]:
var = vars_top[131]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NUMERO_TRABAJADORES_REGISTRADOS_U1M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NUMERO_TRABAJADORES_REGISTRADOS_U1M,missing,32,0.023138,30,2,0.062500,-0.354297,0.002511,0.224288,missing,True
1,NUMERO_TRABAJADORES_REGISTRADOS_U1M,"[-inf,15.0)",578,0.417932,530,48,0.083045,-0.047923,0.000941,0.224288,15.0,False
2,NUMERO_TRABAJADORES_REGISTRADOS_U1M,"[15.0,60.0)",451,0.326103,396,55,0.121951,0.379672,0.054974,0.224288,60.0,False
3,NUMERO_TRABAJADORES_REGISTRADOS_U1M,"[60.0,120.0)",111,0.080260,101,10,0.090090,0.041218,0.000139,0.224288,120.0,False
4,NUMERO_TRABAJADORES_REGISTRADOS_U1M,"[120.0,inf)",211,0.152567,206,5,0.023697,-1.364685,0.165723,0.224288,inf,False


In [232]:
data = add_new_var_x_cut_up(data, colum='DEBT_PORM_u6m_anio_act', cortes=[3000000], impute=0, up=True, soft=False)

>>  _3000000_cut_up_DEBT_PORM_u6m_anio_act
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7074766355140187  PRECISION:  0.06686688455083473  F1:  0.12218545718666773  GINI:  0.10608758849136146
//////////////////////////////////////////////////


In [233]:
var = vars_top[132]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_TRABAJADORES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_TRABAJADORES,missing,32,0.023138,30,2,0.062500,-0.354297,0.002511,0.224288,missing,True
1,NRO_TRABAJADORES,"[-inf,15.0)",578,0.417932,530,48,0.083045,-0.047923,0.000941,0.224288,15.0,False
2,NRO_TRABAJADORES,"[15.0,60.0)",451,0.326103,396,55,0.121951,0.379672,0.054974,0.224288,60.0,False
3,NRO_TRABAJADORES,"[60.0,120.0)",111,0.080260,101,10,0.090090,0.041218,0.000139,0.224288,120.0,False
4,NRO_TRABAJADORES,"[120.0,inf)",211,0.152567,206,5,0.023697,-1.364685,0.165723,0.224288,inf,False


In [234]:
var = vars_top[133]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTDO_COL_DIRECTAS_SF_6UM


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTDO_COL_DIRECTAS_SF_6UM,missing,38,0.027477,38,0,0.000000,-1.396665,0.031625,0.222998,missing,True
1,MTDO_COL_DIRECTAS_SF_6UM,"[-inf,1000000.0)",605,0.437455,554,51,0.084298,-0.039058,0.000656,0.222998,1000000.0,False
2,MTDO_COL_DIRECTAS_SF_6UM,"[1000000.0,1800000.0)",169,0.122198,161,8,0.047337,-0.655681,0.040196,0.222998,1800000.0,False
3,MTDO_COL_DIRECTAS_SF_6UM,"[1800000.0,2200000.0)",84,0.060738,70,14,0.166667,0.736843,0.044487,0.222998,2200000.0,False
4,MTDO_COL_DIRECTAS_SF_6UM,"[2200000.0,2800000.0)",75,0.054230,71,4,0.053333,-0.530104,0.012261,0.222998,2800000.0,False
5,MTDO_COL_DIRECTAS_SF_6UM,"[2800000.0,3600000.0)",96,0.069414,78,18,0.187500,0.879944,0.076666,0.222998,3600000.0,False
6,MTDO_COL_DIRECTAS_SF_6UM,"[3600000.0,5200000.0)",110,0.079537,104,6,0.054545,-0.506350,0.016566,0.222998,5200000.0,False
7,MTDO_COL_DIRECTAS_SF_6UM,"[5200000.0,inf)",206,0.148952,187,19,0.092233,0.059612,0.000542,0.222998,inf,False


In [235]:
var = vars_top[134]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TRX__MTO_CASH_IN_TOTALESTE_MES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TRX__MTO_CASH_IN_TOTALESTE_MES,missing,102,0.073753,92,10,0.098039,0.134550,0.001412,0.221075,missing,True
1,TRX__MTO_CASH_IN_TOTALESTE_MES,"[-inf,500000.0)",816,0.590022,738,78,0.095588,0.106518,0.006996,0.221075,500000.0,False
2,TRX__MTO_CASH_IN_TOTALESTE_MES,"[500000.0,1100000.0)",145,0.104845,126,19,0.131034,0.461910,0.027054,0.221075,1100000.0,False
3,TRX__MTO_CASH_IN_TOTALESTE_MES,"[1100000.0,1500000.0)",71,0.051338,67,4,0.056338,-0.464645,0.009160,0.221075,1500000.0,False
4,TRX__MTO_CASH_IN_TOTALESTE_MES,"[1500000.0,3200000.0)",137,0.099060,135,2,0.014599,-1.858374,0.167666,0.221075,3200000.0,False
5,TRX__MTO_CASH_IN_TOTALESTE_MES,"[3200000.0,inf)",112,0.080983,105,7,0.062500,-0.354297,0.008787,0.221075,inf,False


In [236]:
data = add_new_var_x_cut_up(data, colum='SOW_IBK_MAX_historia', cortes=[0.2, 0.3], impute=0, up=False, soft=True)

>>  _0.2_cut_down_SOW_IBK_MAX_historia
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3532710280373832  PRECISION:  0.0664323374340949  F1:  0.11183431952662722  GINI:  0.05086865982606592
//////////////////////////////////////////////////
>>  _0.3_cut_down_SOW_IBK_MAX_historia
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4803738317757009  PRECISION:  0.07162764771460424  F1:  0.1246665049721077  GINI:  0.101118452064896
//////////////////////////////////////////////////


In [237]:
var = vars_top[135]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MONETARY_VALUE_historia


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MONETARY_VALUE_historia,missing,102,0.073753,92,10,0.098039,0.134550,0.001412,0.221075,missing,True
1,MONETARY_VALUE_historia,"[-inf,500000.0)",816,0.590022,738,78,0.095588,0.106518,0.006996,0.221075,500000.0,False
2,MONETARY_VALUE_historia,"[500000.0,1100000.0)",145,0.104845,126,19,0.131034,0.461910,0.027054,0.221075,1100000.0,False
3,MONETARY_VALUE_historia,"[1100000.0,1500000.0)",71,0.051338,67,4,0.056338,-0.464645,0.009160,0.221075,1500000.0,False
4,MONETARY_VALUE_historia,"[1500000.0,3200000.0)",137,0.099060,135,2,0.014599,-1.858374,0.167666,0.221075,3200000.0,False
5,MONETARY_VALUE_historia,"[3200000.0,inf)",112,0.080983,105,7,0.062500,-0.354297,0.008787,0.221075,inf,False


In [238]:
var = vars_top[136]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TENDENCIA_CASHIN_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TENDENCIA_CASHIN_U12M,missing,102,0.073753,92,10,0.098039,0.134550,0.001412,0.220913,missing,True
1,TENDENCIA_CASHIN_U12M,"[-inf,-40000.0)",73,0.052784,71,2,0.027397,-1.215779,0.048082,0.220913,-40000.0,False
2,TENDENCIA_CASHIN_U12M,"[-40000.0,-14000.0)",101,0.073030,89,12,0.118812,0.350024,0.010337,0.220913,-14000.0,False
3,TENDENCIA_CASHIN_U12M,"[-14000.0,-6000.0)",88,0.063630,85,3,0.034091,-0.990286,0.041889,0.220913,-6000.0,False
4,TENDENCIA_CASHIN_U12M,"[-6000.0,-2000.0)",84,0.060738,77,7,0.083333,-0.044142,0.000116,0.220913,-2000.0,False
5,TENDENCIA_CASHIN_U12M,"[-2000.0,2000.0)",417,0.301518,370,47,0.112710,0.290398,0.028666,0.220913,2000.0,False
6,TENDENCIA_CASHIN_U12M,"[2000.0,4000.0)",87,0.062907,73,14,0.160920,0.702351,0.041346,0.220913,4000.0,False
7,TENDENCIA_CASHIN_U12M,"[4000.0,inf)",431,0.311641,406,25,0.058005,-0.433724,0.049064,0.220913,inf,False


In [239]:
var = vars_top[137]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_DEUDA_FUL_RCCBCP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_DEUDA_FUL_RCCBCP,missing,22,0.015907,21,1,0.045455,-0.690769,0.005729,0.220394,missing,True
1,PERCENT_DEUDA_FUL_RCCBCP,"[-inf,0.1)",822,0.594360,757,65,0.079075,-0.101223,0.005841,0.220394,0.1,False
2,PERCENT_DEUDA_FUL_RCCBCP,"[0.1,0.2)",120,0.086768,100,20,0.166667,0.744315,0.065120,0.220394,0.2,False
3,PERCENT_DEUDA_FUL_RCCBCP,"[0.2,0.30000000000000004)",78,0.056399,75,3,0.038462,-0.865122,0.029745,0.220394,0.30000000000000004,False
4,PERCENT_DEUDA_FUL_RCCBCP,"[0.30000000000000004,0.5)",111,0.080260,93,18,0.162162,0.711526,0.054336,0.220394,0.5,False
5,PERCENT_DEUDA_FUL_RCCBCP,"[0.5,0.8)",120,0.086768,116,4,0.033333,-1.013542,0.059304,0.220394,0.8,False
6,PERCENT_DEUDA_FUL_RCCBCP,"[0.8,inf)",110,0.079537,101,9,0.081818,-0.064143,0.000319,0.220394,inf,False


In [240]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT', cortes=[-2, -5], impute=0, up=False, soft=True)

>>  _-2_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.2672897196261682  PRECISION:  0.06863450923926086  F1:  0.10922283750238686  GINI:  0.04635154360430804
//////////////////////////////////////////////////
>>  _-5_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.19532710280373833  PRECISION:  0.07448325017818959  F1:  0.10784313725490195  GINI:  0.047484679941390384
//////////////////////////////////////////////////


In [241]:
var = vars_top[138]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COVER_GAR_AUTO_HIPO_PREN_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.220342,missing,True
1,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[-inf,0.1)",417,0.301518,394,23,0.055156,-0.487103,0.058593,0.220342,0.1,False
2,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[0.1,1.0)",531,0.383948,482,49,0.092279,0.067630,0.001806,0.220342,1.0,False
3,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[1.0,1.3)",95,0.068691,78,17,0.178947,0.830258,0.066345,0.220342,1.3,False
4,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[1.3,3.2)",170,0.122921,147,23,0.135294,0.498815,0.037549,0.220342,3.2,False
5,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[3.2,inf)",74,0.053507,69,5,0.067568,-0.270915,0.003512,0.220342,inf,False


In [242]:
var = vars_top[139]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTOSALDO_BBVA_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTOSALDO_BBVA_V,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.219316,missing,True
1,MTOSALDO_BBVA_V,"[-inf,100000.0)",737,0.532899,668,69,0.093623,0.083572,0.003853,0.219316,100000.0,False
2,MTOSALDO_BBVA_V,"[100000.0,450000.0)",120,0.086768,117,3,0.025000,-1.309808,0.088591,0.219316,450000.0,False
3,MTOSALDO_BBVA_V,"[450000.0,700000.0)",75,0.054230,69,6,0.080000,-0.088594,0.000410,0.219316,700000.0,False
4,MTOSALDO_BBVA_V,"[700000.0,1800000.0)",124,0.089660,103,21,0.169355,0.763547,0.071352,0.219316,1800000.0,False
5,MTOSALDO_BBVA_V,"[1800000.0,3400000.0)",82,0.059291,76,6,0.073171,-0.185220,0.001884,0.219316,3400000.0,False
6,MTOSALDO_BBVA_V,"[3400000.0,inf)",149,0.107737,137,12,0.080537,-0.081321,0.000689,0.219316,inf,False


In [243]:
var = vars_top[140]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_AUTO_HIPO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_AUTO_HIPO,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.21832,missing,True
1,PERCENT_GAR_AUTO_HIPO,"[-inf,0.30000000000000004)",858,0.620390,788,70,0.081585,-0.067249,0.002729,0.21832,0.30000000000000004,False
2,PERCENT_GAR_AUTO_HIPO,"[0.30000000000000004,0.4)",105,0.075922,85,20,0.190476,0.906834,0.090109,0.21832,0.4,False
3,PERCENT_GAR_AUTO_HIPO,"[0.4,0.6000000000000001)",130,0.093999,126,4,0.030769,-1.096234,0.072822,0.21832,0.6000000000000001,False
4,PERCENT_GAR_AUTO_HIPO,"[0.6000000000000001,0.8)",143,0.103398,130,13,0.090909,0.051168,0.000277,0.21832,0.8,False
5,PERCENT_GAR_AUTO_HIPO,"[0.8,inf)",100,0.072307,88,12,0.120000,0.361323,0.010957,0.21832,inf,False


In [244]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_PRIN_DEUDASF_ULT_RCC', cortes=[3, 5], impute=0, up=True, soft=False)

>>  _3_cut_up_NUM_BANCOS_PRIN_DEUDASF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5588785046728972  PRECISION:  0.08324053452115814  F1:  0.1448994426944512  GINI:  0.18394966486873
//////////////////////////////////////////////////
>>  _5_cut_up_NUM_BANCOS_PRIN_DEUDASF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09906542056074766  PRECISION:  0.11312700106723586  F1:  0.10563029397110114  GINI:  0.05175812237106303
//////////////////////////////////////////////////


In [245]:
var = vars_top[141]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_PREN_WAR


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_PREN_WAR,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.217164,missing,True
1,PERCENT_GAR_PREN_WAR,"[-inf,0.01)",916,0.662328,852,64,0.069869,-0.234950,0.033187,0.217164,0.01,False
2,PERCENT_GAR_PREN_WAR,"[0.01,0.08)",179,0.129429,165,14,0.078212,-0.113135,0.001581,0.217164,0.08,False
3,PERCENT_GAR_PREN_WAR,"[0.08,inf)",241,0.174259,200,41,0.170124,0.769008,0.140970,0.217164,inf,False


In [246]:
var = vars_top[142]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT_IBK


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT_IBK,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.216905,missing,True
1,GAR_TOT_IBK,"[-inf,200000.0)",491,0.355025,440,51,0.103870,0.198804,0.015233,0.216905,200000.0,False
2,GAR_TOT_IBK,"[200000.0,400000.0)",135,0.097614,130,5,0.037037,-0.904343,0.055403,0.216905,400000.0,False
3,GAR_TOT_IBK,"[400000.0,2600000.0)",385,0.278380,348,37,0.096104,0.112469,0.003689,0.216905,2600000.0,False
4,GAR_TOT_IBK,"[2600000.0,4200000.0)",121,0.087491,113,8,0.066116,-0.294193,0.006708,0.216905,4200000.0,False
5,GAR_TOT_IBK,"[4200000.0,6400000.0)",102,0.073753,99,3,0.029412,-1.142754,0.061006,0.216905,6400000.0,False
6,GAR_TOT_IBK,"[6400000.0,inf)",102,0.073753,87,15,0.147059,0.595895,0.033440,0.216905,inf,False


In [247]:
var = vars_top[143]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COSTO_VENTAS_VAR_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COSTO_VENTAS_VAR_CEF_SITUACION,missing,285,0.206074,261,24,0.084211,-0.032713,0.000218,0.216016,missing,True
1,COSTO_VENTAS_VAR_CEF_SITUACION,"[-inf,-0.20000000000000018)",441,0.318872,384,57,0.129252,0.446162,0.076277,0.216016,-0.20000000000000018,False
2,COSTO_VENTAS_VAR_CEF_SITUACION,"[-0.20000000000000018,-2.220446049250313e-16)",233,0.168474,214,19,0.081545,-0.067784,0.000753,0.216016,-2.220446049250313e-16,False
3,COSTO_VENTAS_VAR_CEF_SITUACION,"[-2.220446049250313e-16,0.19999999999999973)",207,0.149675,198,9,0.043478,-0.737289,0.060288,0.216016,0.19999999999999973,False
4,COSTO_VENTAS_VAR_CEF_SITUACION,"[0.19999999999999973,0.49999999999999956)",89,0.064353,87,2,0.022472,-1.419008,0.074096,0.216016,0.49999999999999956,False
5,COSTO_VENTAS_VAR_CEF_SITUACION,"[0.49999999999999956,inf)",128,0.092552,119,9,0.070312,-0.228146,0.004385,0.216016,inf,False


In [248]:
var = vars_top[144]
print(">>>>> ", var)
display(cortes[var])

>>>>>  VENTAS_NETAS_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,VENTAS_NETAS_CEF_SITUACION,missing,285,0.206074,261,24,0.084211,-0.032713,0.000218,0.215673,missing,True
1,VENTAS_NETAS_CEF_SITUACION,"[-inf,1000.0)",137,0.099060,122,15,0.109489,0.257783,0.007322,0.215673,1000.0,False
2,VENTAS_NETAS_CEF_SITUACION,"[1000.0,3000.0)",222,0.160521,190,32,0.144144,0.572465,0.066538,0.215673,3000.0,False
3,VENTAS_NETAS_CEF_SITUACION,"[3000.0,23000.0)",651,0.470716,603,48,0.073733,-0.176963,0.013703,0.215673,23000.0,False
4,VENTAS_NETAS_CEF_SITUACION,"[23000.0,inf)",88,0.063630,87,1,0.011364,-2.112155,0.127892,0.215673,inf,False


In [249]:
var = vars_top[145]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTOSALDO_OTROS_BANCOS_PRIN_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTOSALDO_OTROS_BANCOS_PRIN_V,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.211874,missing,True
1,MTOSALDO_OTROS_BANCOS_PRIN_V,"[-inf,1000000.0)",646,0.467101,597,49,0.075851,-0.146343,0.009417,0.211874,1000000.0,False
2,MTOSALDO_OTROS_BANCOS_PRIN_V,"[1000000.0,1800000.0)",153,0.110629,128,25,0.163399,0.720599,0.077095,0.211874,1800000.0,False
3,MTOSALDO_OTROS_BANCOS_PRIN_V,"[1800000.0,2200000.0)",77,0.055676,72,5,0.064935,-0.313475,0.004809,0.211874,2200000.0,False
4,MTOSALDO_OTROS_BANCOS_PRIN_V,"[2200000.0,4200000.0)",181,0.130875,161,20,0.110497,0.268081,0.010507,0.211874,4200000.0,False
5,MTOSALDO_OTROS_BANCOS_PRIN_V,"[4200000.0,5600000.0)",75,0.054230,71,4,0.053333,-0.522632,0.011959,0.211874,5600000.0,False
6,MTOSALDO_OTROS_BANCOS_PRIN_V,"[5600000.0,10000000.0)",80,0.057845,69,11,0.137500,0.517542,0.019167,0.211874,10000000.0,False
7,MTOSALDO_OTROS_BANCOS_PRIN_V,"[10000000.0,inf)",75,0.054230,72,3,0.040000,-0.824300,0.026383,0.211874,inf,False


In [250]:
var = vars_top[146]
print(">>>>> ", var)
display(cortes[var])

>>>>>  IMPORTE_IMPAG_U6M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,IMPORTE_IMPAG_U6M,missing,753,0.544469,708,45,0.059761,-0.402028,0.074604,0.211082,missing,True
1,IMPORTE_IMPAG_U6M,"[-inf,50.0)",288,0.208243,246,42,0.145833,0.586091,0.090976,0.211082,50.0,False
2,IMPORTE_IMPAG_U6M,"[50.0,500.0)",136,0.098337,118,18,0.132353,0.473441,0.026783,0.211082,500.0,False
3,IMPORTE_IMPAG_U6M,"[500.0,3350.0)",126,0.091106,119,7,0.055556,-0.479460,0.017206,0.211082,3350.0,False
4,IMPORTE_IMPAG_U6M,"[3350.0,inf)",80,0.057845,72,8,0.100000,0.156529,0.001512,0.211082,inf,False


In [251]:
var = vars_top[147]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ratio_PROM_CANT_TRX_U6M_U12M_CASHIN


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ratio_PROM_CANT_TRX_U6M_U12M_CASHIN,missing,102,0.073753,92,10,0.098039,0.134550,0.001412,0.208229,missing,True
1,ratio_PROM_CANT_TRX_U6M_U12M_CASHIN,"[-inf,0.6000000000000001)",176,0.127260,151,25,0.142045,0.555349,0.049302,0.208229,0.6000000000000001,False
2,ratio_PROM_CANT_TRX_U6M_U12M_CASHIN,"[0.6000000000000001,1.0)",340,0.245842,306,34,0.100000,0.156529,0.006426,0.208229,1.0,False
3,ratio_PROM_CANT_TRX_U6M_U12M_CASHIN,"[1.0,1.3)",418,0.302242,399,19,0.045455,-0.690769,0.108852,0.208229,1.3,False
4,ratio_PROM_CANT_TRX_U6M_U12M_CASHIN,"[1.3,1.6)",159,0.114967,140,19,0.119497,0.356550,0.016931,0.208229,1.6,False
5,ratio_PROM_CANT_TRX_U6M_U12M_CASHIN,"[1.6,2.0)",74,0.053507,71,3,0.040541,-0.810314,0.025294,0.208229,2.0,False
6,ratio_PROM_CANT_TRX_U6M_U12M_CASHIN,"[2.0,inf)",114,0.082430,104,10,0.087719,0.011948,0.000012,0.208229,inf,False


In [252]:
var = vars_top[148]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DIFF_CASHIN_OVER_CASHOUT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DIFF_CASHIN_OVER_CASHOUT,missing,188,0.135936,166,22,0.117021,0.332808,0.017273,0.204435,missing,True
1,DIFF_CASHIN_OVER_CASHOUT,"[-inf,-90000.0)",203,0.146782,193,10,0.049261,-0.606352,0.042128,0.204435,-90000.0,False
2,DIFF_CASHIN_OVER_CASHOUT,"[-90000.0,-20000.0)",122,0.088214,105,17,0.139344,0.533006,0.031198,0.204435,-20000.0,False
3,DIFF_CASHIN_OVER_CASHOUT,"[-20000.0,0.0)",244,0.176428,226,18,0.073770,-0.176410,0.005105,0.204435,0.0,False
4,DIFF_CASHIN_OVER_CASHOUT,"[0.0,10000.0)",194,0.140275,171,23,0.118557,0.347584,0.019560,0.204435,10000.0,False
5,DIFF_CASHIN_OVER_CASHOUT,"[10000.0,20000.0)",90,0.065076,87,3,0.033333,-1.013542,0.044478,0.204435,20000.0,False
6,DIFF_CASHIN_OVER_CASHOUT,"[20000.0,50000.0)",117,0.084599,103,14,0.119658,0.358082,0.012574,0.204435,50000.0,False
7,DIFF_CASHIN_OVER_CASHOUT,"[50000.0,190000.0)",126,0.091106,120,6,0.047619,-0.641979,0.028897,0.204435,190000.0,False
8,DIFF_CASHIN_OVER_CASHOUT,"[190000.0,inf)",99,0.071584,92,7,0.070707,-0.222125,0.003223,0.204435,inf,False


In [253]:
var = vars_top[149]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_PREN_WAR_TOT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_PREN_WAR_TOT,missing,47,0.033984,46,1,0.021277,-1.474888,0.041426,0.204406,missing,True
1,GAR_PREN_WAR_TOT,"[-inf,100000.0)",918,0.663774,857,61,0.066449,-0.288811,0.049158,0.204406,100000.0,False
2,GAR_PREN_WAR_TOT,"[100000.0,320000.0)",141,0.101952,125,16,0.113475,0.298028,0.010241,0.204406,320000.0,False
3,GAR_PREN_WAR_TOT,"[320000.0,inf)",277,0.200289,235,42,0.151625,0.631837,0.103580,0.204406,inf,False


In [254]:
display(cortes['NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC,missing,96,0.069414,93,3,0.031250,-1.080234,0.052536,0.055025,missing,True
1,NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC,"[-inf,inf)",1287,0.930586,1170,117,0.090909,0.051168,0.002489,0.055025,inf,False


In [255]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC', cortes=[1, 2], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1560747663551402  PRECISION:  0.2014475271411339  F1:  0.17588204318062142  GINI:  0.11838832664205823
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.014953271028037384  PRECISION:  0.43243243243243246  F1:  0.028906955736224028  GINI:  0.013757779738045262
//////////////////////////////////////////////////


In [256]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN', cortes=[-19, -5], impute=0, up=False, soft=True)

>>  _-19_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04953271028037383  PRECISION:  0.11300639658848614  F1:  0.06887589343729694  GINI:  0.02585059710719828
//////////////////////////////////////////////////
>>  _-5_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.102803738317757  PRECISION:  0.09385665529010238  F1:  0.09812667261373773  GINI:  0.04234603593816
//////////////////////////////////////////////////


In [257]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN', cortes=[30], impute=0, up=True, soft=False)

>>  _30_cut_up_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.02336448598130841  PRECISION:  0.044169611307420496  F1:  0.03056234718826406  GINI:  -0.007433646775152902
//////////////////////////////////////////////////


In [258]:
data = add_new_var_x_cut_up(data, colum='TRX_TIPOS_CASH_OUT_CAIDA_10PERCENT_MES_ANT', cortes=[1,2,3], impute=0, up=True, soft=False)

>>  _1_cut_up_TRX_TIPOS_CASH_OUT_CAIDA_10PERCENT_MES_ANT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5046728971962616  PRECISION:  0.05623828369089773  F1:  0.10119940029985006  GINI:  -0.011210058513632548
//////////////////////////////////////////////////
>>  _2_cut_up_TRX_TIPOS_CASH_OUT_CAIDA_10PERCENT_MES_ANT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.24205607476635513  PRECISION:  0.059953703703703703  F1:  0.09610389610389611  GINI:  0.01087083054456306
//////////////////////////////////////////////////
>>  _3_cut_up_TRX_TIPOS_CASH_OUT_CAIDA_10PERCENT_MES_ANT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08785046728971962  PRECISION:  0.06451612903225806  F1:  0.074396517609814  GINI:  0.010257389753570267
//////////////////////////////////////////////////


In [259]:
data = add_new_var_x_cut_up(data, colum='COVER_GAR_AUTO_HIPO_ULT_RCC', cortes=[0.4, 0.5, 0.6, 0.7], impute=0, up=False, soft=True)

>>  _0.4_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6289719626168224  PRECISION:  0.052992125984251966  F1:  0.09774872912127815  GINI:  -0.05570297760861309
//////////////////////////////////////////////////
>>  _0.5_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.677570093457944  PRECISION:  0.05426240550856972  F1:  0.10047813734321945  GINI:  -0.04177409417726041
//////////////////////////////////////////////////
>>  _0.6_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7261682242990655  PRECISION:  0.05570291777188329  F1:  0.10346893934349824  GINI:  -0.023689455309268803
//////////////////////////////////////////////////
>>  _0.7_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7728971962616823  PRECISION:  0.05724768101896719  F1:  0.1065996390822377  GINI:  -0.00240

In [260]:
#data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO_SFSF_ULT_RCC', cortes=[1, 2, 3, 4, 5], impute=0, up=True, soft=False)

In [261]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF', cortes=[1, 2, 3, 4], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.12429906542056075  PRECISION:  0.18046132971506107  F1:  0.14720531267293857  GINI:  0.08991445879412341
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09252336448598131  PRECISION:  0.24688279301745636  F1:  0.13460231135282122  GINI:  0.07533106117276245
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.05794392523364486  PRECISION:  0.29523809523809524  F1:  0.09687500000000002  GINI:  0.04951855804703453
//////////////////////////////////////////////////
>>  _4_cut_up_NUM_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.03364485981308411  PRECISION

In [262]:
#data = add_new_var_x_cut_up(data, colum='VARACION_NUM_BANCOS_FULL_RCCC_SF', cortes=[1], impute=0, up=True, soft=False)

In [263]:
[col for col in data.columns if 'VARACION_NUM_BANCOS_FULL_RCCC_SF' in col.upper() if 'and' not in col]

['VARACION_NUM_BANCOS_FULL_RCCC_SF']

In [264]:
data = add_new_var_x_cut_up(data, colum='PERCENT_COL_DIRECTAS_SF', cortes=[0.7, 0.89, 0.99], impute=0, up=True, soft=False)

>>  _0.7_cut_up_PERCENT_COL_DIRECTAS_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.8682242990654205  PRECISION:  0.0597427652733119  F1:  0.11179302045728039  GINI:  0.03587772044763615
//////////////////////////////////////////////////
>>  _0.89_cut_up_PERCENT_COL_DIRECTAS_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7317757009345794  PRECISION:  0.05877937091809924  F1:  0.10881801125703565  GINI:  0.018010472652671305
//////////////////////////////////////////////////
>>  _0.99_cut_up_PERCENT_COL_DIRECTAS_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.514018691588785  PRECISION:  0.0549615269311482  F1:  0.09930486593843098  GINI:  -0.024350886004292693
//////////////////////////////////////////////////


In [265]:
display(cortes['NRO_CLASIF_NO_NORMAL_GRUPO_EECON'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_CLASIF_NO_NORMAL_GRUPO_EECON,missing,132,0.095445,119,13,0.098485,0.139579,0.001970,0.002192,missing,True
1,NRO_CLASIF_NO_NORMAL_GRUPO_EECON,"[-inf,inf)",1251,0.904555,1144,107,0.085532,-0.015704,0.000222,0.002192,inf,False


In [266]:
display(cortes['NRO_FEVE_DURO_GRUPO_EECON'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_FEVE_DURO_GRUPO_EECON,missing,132,0.095445,119,13,0.098485,0.139579,0.001970,0.002192,missing,True
1,NRO_FEVE_DURO_GRUPO_EECON,"[-inf,inf)",1251,0.904555,1144,107,0.085532,-0.015704,0.000222,0.002192,inf,False


In [267]:
data = add_new_var_x_cut_up(data, colum='NRO_CLASIF_NO_NORMAL_GRUPO_EECON', cortes=[1], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_CLASIF_NO_NORMAL_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.05700934579439252  PRECISION:  0.08603667136812412  F1:  0.06857785272625071  GINI:  0.020119900274638614
//////////////////////////////////////////////////


In [268]:
data = add_new_var_x_cut_up(data, colum='NRO_FEVE_DURO_GRUPO_EECON', cortes=[1], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_FEVE_DURO_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.016822429906542057  PRECISION:  0.09183673469387756  F1:  0.028436018957345974  GINI:  0.006689218019942933
//////////////////////////////////////////////////


In [269]:
data = add_new_var_x_cut_up(data, colum='NRO_FEVE_DURO_GRUPO_EECON', cortes=[1], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_FEVE_DURO_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.016822429906542057  PRECISION:  0.09183673469387756  F1:  0.028436018957345974  GINI:  0.006689218019942933
//////////////////////////////////////////////////


## OTRAS VARIABLES

In [270]:
"""
for var in [col for col in data.columns if 'full' in col.lower()]:
    try:
        print(">"*20, ' ', var)
        display(cortes[var])
    except Exception as e:
        print(str(e))
        """

'\nfor var in [col for col in data.columns if \'full\' in col.lower()]:\n    try:\n        print(">"*20, \' \', var)\n        display(cortes[var])\n    except Exception as e:\n        print(str(e))\n        '

In [271]:
#data['VARIAICION_NRO_BANCOS_CIERRE_2019'].value_counts()

In [272]:

data = add_new_var_x_cut_up(data, colum='TENDENCIA_PROTESTOS_U7M', cortes=[0], impute=0, up=True, soft=True)
data = add_new_var_x_cut_up(data, colum='NRO_MESES_ANTERIORES_CON_PROTESTOS_CONSECUTIVOS', cortes=[2], impute=0, up=True, soft=False)

>>  _0_cut_up_TENDENCIA_PROTESTOS_U7M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.03457943925233645  PRECISION:  0.06788990825688074  F1:  0.045820433436532505  GINI:  0.005659935665862648
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_MESES_ANTERIORES_CON_PROTESTOS_CONSECUTIVOS
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04579439252336449  PRECISION:  0.0831918505942275  F1:  0.0590717299578059  GINI:  0.015053187923569578
//////////////////////////////////////////////////


In [273]:
data = add_new_var_x_cut_up(data, colum='M_Septile', cortes=[5], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='F_Septile', cortes=[5], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='M_Septile', cortes=[6], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='F_Septile', cortes=[6], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='M_Septile', cortes=[7], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='F_Septile', cortes=[7], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='F_Quartile', cortes=[4], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='M_Quartile', cortes=[4], impute=0, up=True, soft=False)

data = add_new_var_x_cut_up(data, colum='NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC', cortes=[3,4,5,6], impute=0, up=True, soft=False)
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_CON_MAYOR_SOW_ULT_RCC', cortes=[1, 2], impute=0, up=True, soft=False)

data = add_new_var_x_cut_up(data, colum='TENDENCIA_PROTESTOS_U7M', cortes=[0], impute=0, up=True, soft=True)
data = add_new_var_x_cut_up(data, colum='NRO_MESES_ANTERIORES_CON_PROTESTOS_CONSECUTIVOS', cortes=[2], impute=0, up=True, soft=False)

>>  _5_cut_up_M_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4457943925233645  PRECISION:  0.060257705912076805  F1:  0.10616514578232807  GINI:  0.02230583508285444
//////////////////////////////////////////////////
>>  _5_cut_up_F_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.44672897196261685  PRECISION:  0.059757469683710464  F1:  0.10541404785533136  GINI:  0.01857230567547119
//////////////////////////////////////////////////
>>  _6_cut_up_M_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.28785046728971964  PRECISION:  0.059128431560760225  F1:  0.09810479375696768  GINI:  0.008845571468246272
//////////////////////////////////////////////////
>>  _6_cut_up_F_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.29439252336448596  PRECISION:  0.05831173639392818  F1:  0.0973423980222497  GINI:  0.004798990403083181
//////////////////////////////////////////////////
>>  _7_cut_up_M_Septi

In [274]:
for column in [col for col in data.columns if 'TENDENCIA' in col.upper() and 'and' not in col]:
    try:
        data = add_new_var_x_cut_up(data, colum=column, cortes=[0], impute=0, up=True, soft=True)
    except:
        print(">>ERROR: ", column)

>>  _0_cut_up_TENDENCIA_PERCENT_NO_CLASIF_NORMAL_GRUPO_ECON_U7M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.037383177570093455  PRECISION:  0.07561436672967864  F1:  0.050031269543464665  GINI:  0.009545308960279053
//////////////////////////////////////////////////
>>  _0_cut_up_GRUPO_ECON_TIENE_TENDENCIA_AUMENTO_NO_CLASIF_NORMAL_U7M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.037383177570093455  PRECISION:  0.07561436672967864  F1:  0.050031269543464665  GINI:  0.009545308960279053
//////////////////////////////////////////////////
>>  _0_cut_up_TENDENCIA_PERCENT_NO_CLASIF_NORMAL_GRUPO_ECON_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.013084112149532711  PRECISION:  0.06451612903225806  F1:  0.021756021756021756  GINI:  0.0015276963462764748
//////////////////////////////////////////////////
>>  _0_cut_up_GRUPO_ECON_TIENE_TENDENCIA_AUMENTO_NO_CLASIF_NORMAL_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  

### IMPUTANDO VARIABLES DICOTOMIAS

In [275]:
for col in [_ for _ in data.columns if 'TIENE' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'CUT' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'MENOR' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'MAYOR' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'SUBIDA' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'NUM' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'INT' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'CAIDA' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'NRO' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'COVID' in _.upper()]:
    data[col] = data[col].fillna(0)

### SAVE

In [280]:
import gc
gc.collect()
print(data.shape)

(24014, 2918)


In [281]:
print(data.shape)
data = data[[col for col in data.columns if '1_and_1' not in col.lower() and col.count('cut_up') <= 1]]

print(data.shape)
data.to_csv("AVANCE_6_S2_PROVINCIA.csv", index=False)
#data.to_excel("AVANCE_6_S2_PROVINCIA.xlsx", index=False)

(24014, 2918)
(24014, 2918)
